# **Question Answering System**

* **Pre-Trained Transformer Model: ALBERT**

* **Dataset: SQuAD 2.0**

* **Transformer Library: Huggings Face**

ALBERT has shorter training and inference times than BERT. We would be fine tuning ALBERT for our Q&A task.

## Check GPU Settings

In [2]:
!nvidia-smi

Tue Jun 14 09:41:36 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
    raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


## Clone hugginghace transformers library from Github

In [4]:
!git clone https://github.com/huggingface/transformers \
&& cd transformers \
&& git checkout a3085020ed0d81d4903c50967687192e3101e770 

Cloning into 'transformers'...
remote: Enumerating objects: 97968, done.
remote: Counting objects: 100% (163/163), done.
remote: Compressing objects: 100% (101/101), done.
remote: Total 97968 (delta 69), reused 105 (delta 46), pack-reused 97805
Receiving objects: 100% (97968/97968), 91.45 MiB | 11.35 MiB/s, done.
Resolving deltas: 100% (72070/72070), done.
Note: checking out 'a3085020ed0d81d4903c50967687192e3101e770'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by performing another checkout.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -b with the checkout command again. Example:

  git checkout -b <new-branch-name>

HEAD is now at a3085020e Added repetition penalty to PPLM example (#2436)


## Install ``transformers`` and ``tensorboardX``

In [5]:
!pip install ./transformers/
!pip install tensorboardX

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Processing ./transformers
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
     |████████████████████████████████| 4.7 MB 4.2 MB/s 
     |████████████████████████████████| 132 kB 72.9 MB/s 
     |████████████████████████████████| 1.2 MB 63.6 MB/s 
     |████████████████████████████████| 880 kB 68.3 MB/s 
     |████████████████████████████████| 79 kB 9.5 MB/s 
     |████████████████████████████████| 8.9 MB 53.3 MB/s 
     |████████████████████████████████| 138 kB 81.5 MB/s 
     |████████████████████████████████| 127 kB 81.7 MB/s 
  Created wheel fo

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 125 kB 4.2 MB/s 


## Get Training Data SQuAD 2.0

In [6]:
!mkdir dataset \
&& cd dataset \
&& wget https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v2.0.json \
&& wget https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v2.0.json

--2022-06-14 09:42:15--  https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v2.0.json
Resolving rajpurkar.github.io (rajpurkar.github.io)... 185.199.108.153, 185.199.109.153, 185.199.110.153, ...
Connecting to rajpurkar.github.io (rajpurkar.github.io)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 42123633 (40M) [application/json]
Saving to: ‘train-v2.0.json’

train-v2.0.json     100%[===================>]  40.17M   169MB/s    in 0.2s    

2022-06-14 09:42:19 (169 MB/s) - ‘train-v2.0.json’ saved [42123633/42123633]

--2022-06-14 09:42:19--  https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v2.0.json
Resolving rajpurkar.github.io (rajpurkar.github.io)... 185.199.108.153, 185.199.109.153, 185.199.110.153, ...
Connecting to rajpurkar.github.io (rajpurkar.github.io)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4370528 (4.2M) [application/json]
Saving to: ‘dev-v2.0.json’

dev-v2.0.json       1

## Train the model ALBERT 

In [7]:
!export SQUAD_DIR=/content/dataset \
&& python transformers/examples/run_squad.py \
  --model_type albert \
  --model_name_or_path albert-base-v2 \
  --do_train \
  --do_eval \
  --do_lower_case \
  --train_file $SQUAD_DIR/train-v2.0.json \
  --predict_file $SQUAD_DIR/dev-v2.0.json \
  --per_gpu_train_batch_size 12 \
  --learning_rate 3e-5 \
  --num_train_epochs 1.0 \
  --max_seq_length 384 \
  --doc_stride 128 \
  --output_dir /content/model_output \
  --save_steps 1000 \
  --threads 4 \
  --version_2_with_negative 

Die letzten 5000 Zeilen der Streamingausgabe wurden abgeschnitten.
Iteration:  56% 6181/11017 [47:46<37:19,  2.16it/s]
Iteration:  56% 6182/11017 [47:46<37:20,  2.16it/s]
Iteration:  56% 6183/11017 [47:47<37:20,  2.16it/s]
Iteration:  56% 6184/11017 [47:47<37:20,  2.16it/s]
Iteration:  56% 6185/11017 [47:48<37:19,  2.16it/s]
Iteration:  56% 6186/11017 [47:48<37:18,  2.16it/s]
Iteration:  56% 6187/11017 [47:49<37:18,  2.16it/s]
Iteration:  56% 6188/11017 [47:49<37:17,  2.16it/s]
Iteration:  56% 6189/11017 [47:49<37:16,  2.16it/s]
Iteration:  56% 6190/11017 [47:50<37:15,  2.16it/s]
Iteration:  56% 6191/11017 [47:50<37:16,  2.16it/s]
Iteration:  56% 6192/11017 [47:51<37:19,  2.15it/s]
Iteration:  56% 6193/11017 [47:51<37:18,  2.15it/s]
Iteration:  56% 6194/11017 [47:52<37:19,  2.15it/s]
Iteration:  56% 6195/11017 [47:52<37:17,  2.16it/s]
Iteration:  56% 6196/11017 [47:53<37:17,  2.15it/s]
Iteration:  56% 6197/11017 [47:53<37:17,  2.15it/s]
Iteration:  56% 6198/11017 [47:54<37:16,  2.15it/

In [15]:
# NOTE:
## Uncomment and run the following code, if you get an error like "cannot import name 'PROTOCOL_TLS' from 'urllib3.util.ssl_'"
## An incompatible version of awscli can cause this error
## To fix this, you need to reinstall awscli and restart the runtime
## Source: https://www.datasciencelearner.com/importerror-cannot-import-name-ssl-from-urllib3-util-ssl_-fix/

# pip install awscli --ignore-installed six

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 3.9 MB 4.1 MB/s 
     |████████████████████████████████| 548 kB 68.9 MB/s 
  Using cached s3transfer-0.6.0-py3-none-any.whl (79 kB)
  Using cached botocore-1.27.8-py3-none-any.whl (8.9 MB)
     |████████████████████████████████| 636 kB 67.0 MB/s 
     |████████████████████████████████| 247 kB 99.1 MB/s 
  Using cached urllib3-1.26.9-py2.py3-none-any.whl (138 kB)
  Using cached jmespath-1.0.0-py3-none-any.whl (23 kB)
     |████████████████████████████████| 77 kB 6.3 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.9 which is incompatible.
google-colab 1.0.0 requires six~=1.15.0, but you have six 1.16.0 which is incompatible.
datas

<font color="#ffb142">model_name_or_path = "ktrapeznikov/albert-xlarge-v2-squad-v2"

In [12]:
import os
import torch
import time
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

from transformers import (
    AlbertConfig,
    AlbertForQuestionAnswering,
    AlbertTokenizer,
    squad_convert_examples_to_features
)

from transformers.data.processors.squad import SquadResult, SquadV2Processor, SquadExample

from transformers.data.metrics.squad_metrics import compute_predictions_logits

# READER NOTE: Set this flag to use own model, or use pretrained model in the Hugging Face repository
use_own_model = False

if use_own_model:
  model_name_or_path = "/content/model_output"
else:
  model_name_or_path = "ktrapeznikov/albert-xlarge-v2-squad-v2"

output_dir = ""


## Configure QA model

In [13]:
#Configuring A Question Answer Model
# Config
#The number of predictions given per question.
n_best_size = 1
#The maximum token length of an answer that can be generated.
max_answer_length = 30
do_lower_case = True
#If (null_score - best_non_null) is greater than the threshold predict null.
null_score_diff_threshold = 0.0

## Setup model and tensor attributes

In [14]:
# Returns a new Tensor
def to_list(tensor):
    return tensor.detach().cpu().tolist()

In [15]:
# Setup model
config_class, model_class, tokenizer_class = (
    AlbertConfig, AlbertForQuestionAnswering, AlbertTokenizer
)
config = config_class.from_pretrained(model_name_or_path)

tokenizer = tokenizer_class.from_pretrained(model_name_or_path, do_lower_case=True)

model = model_class.from_pretrained(model_name_or_path, config=config)

In [16]:
#tensor attributes 
#A torch.device is an object representing the device on which a torch.Tensor is or will be allocated
#the Tensor.device property allows a torch.Tensor device to be accessed
#The torch.device contains a device type either ('cpu' or 'cuda') 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model.to(device)

processor = SquadV2Processor()

## Create prediction function

In [17]:
def run_prediction(question_texts, context_text):
    """Setup function to compute predictions"""
    examples = []

    for i, question_text in enumerate(question_texts):
        example = SquadExample(
            qas_id=str(i),
            question_text=question_text,
            context_text=context_text,
            answer_text=None,
            start_position_character=None,
            title="Predict",
            is_impossible=False,
            answers=None,
        )

        examples.append(example)

    features, dataset = squad_convert_examples_to_features(
        examples=examples,
        tokenizer=tokenizer,
        max_seq_length=384,
        doc_stride=128,
        max_query_length=64,
        is_training=False,
        return_dataset="pt",
        threads=1,
    )

    eval_sampler = SequentialSampler(dataset)
    eval_dataloader = DataLoader(dataset, sampler=eval_sampler, batch_size=10)

    all_results = []

    for batch in eval_dataloader:
        model.eval()
        batch = tuple(t.to(device) for t in batch)

        with torch.no_grad():
            inputs = {
                "input_ids": batch[0],
                "attention_mask": batch[1],
                "token_type_ids": batch[2],
            }

            example_indices = batch[3]

            outputs = model(**inputs)

            for i, example_index in enumerate(example_indices):
                eval_feature = features[example_index.item()]
                unique_id = int(eval_feature.unique_id)

                output = [to_list(output[i]) for output in outputs]

                start_logits, end_logits = output
                result = SquadResult(unique_id, start_logits, end_logits)
                all_results.append(result)

    output_prediction_file = "predictions.json"
    output_nbest_file = "nbest_predictions.json"
    output_null_log_odds_file = "null_predictions.json"

    predictions = compute_predictions_logits(
        examples,
        features,
        all_results,
        n_best_size,
        max_answer_length,
        do_lower_case,
        output_prediction_file,
        output_nbest_file,
        output_null_log_odds_file,
        False,  # verbose_logging
        True,  # version_2_with_negative
        null_score_diff_threshold,
        tokenizer,
    )

    return predictions

## Example 1 - Aircraft type training manual

### Define context and question

In [42]:
context1 = """The reverse thrust is controlled by the ECU. 
A manual selection of the reverse is done when the flight crew lifts the latching levers on the throttle control levers. 
The reverse thrust command is sent to the ECU and the Engine Interface Unit (EIU). 
The signal from the ECU to the directional valve is supplied to an inhibition relay controlled by the 
Engine Interface Unit (EIU) in relation to the position of the throttle control lever. 
In relation to commands from the ECU, a Hydraulic Control Unit (HCU) supplies hydraulic power to operate the thrust reverser. 
The thrust reverser uses 4 hydraulically operated pivoting blocker doors to redirect the engine fan airflow."""

question1 = ["Which component supplies hydraulic power to the thrust reverser?"]


### Generate and print prediction

In [43]:
prediction1 = run_prediction(question1, context1)

for key in prediction1.keys():
  print(prediction1[key])

add example index and unique id: 100%|██████████| 1/1 [00:00<00:00, 10485.76it/s]


Hydraulic Control Unit (HCU)


## Example 2 - Flight crew operation manual

### Define context and question

In [40]:
context2 = """

A®318/A319/A320/A321
TURKISH AIRLINES

FLIGHT CREW OPERATING MANUAL
FUEL
CONTENTS
1.28.00	p 1


SEQ 001 I REV 36

28.00	CONTENTS

R 28.10	DESCRIPTION
- GENERAL	1
- TANKS	1
R	- ENGINE FEED	2
R	- APU FEED	6
R	- FUEL RECIRCULATION SYSTEM	6
R	- REFUELING AND DEFUELING	7
R	- FUEL QUANTITY INDICATION AND LEVEL SENSING	8
28.20	CONTROLS AND INDICATORS
- OVERHEAD PANEL	1
- REFUELING CONTROL PANEL	3
- MAINTENANCE PANEL	6
- ECAM FUEL PAGE	8
- ECAM UPPER DISPLAY	11
- WARNINGS AND CAUTIONS	12
- MEMO DISPLAY	13

28.30	ELECTRICAL SUPPLY
- BUS EQUIPMENT LIST























THY ALL

- circulates fuel to cool the integrated drive generator (IDG)
- keeps fuel in the outer wing for wing bending and flutter relief.

I TANKS I

The fuel is stored in the wings and the center section. The wings have inner and outer tanks.
There is a vent surge tank outboard of the outer tank in each wing.
When the aircraft has been refueled to maximum capacity, the fuel can expand by 2 % (20° temperature rise) without spilling.
There is an overpressure protector in each vent outer and inner tank and between the center tank and the left inner tank.
INNER CENTER INNER TANK TANK TANK


<
<
0"'
N
<
I
0
0
I
0
.,
N
I
0
I
"u'

2

USABLE FUEL

OUTER TANKS
INNER TANKS
CENTER TANK
TOTAL

VOLUME
(liters)
880 X 2
6924 X 2
8250
23858

(US gallons)
232 X 2
1829 X 2
2180
6302

WEIGHT*
(KG)
691 X 2
5435 X 2
6476
18728

(LB)
1520 X 2
11982 X 2
14281
41285

* Fuel density : 0.785 kg/I or 6.551 lb/US Gal.

THY MSN 1856-2739 2928-2984 3010 3142-3185 3208-3257 3303-3341 3558-3603 3654 3718-3719
4629


The main fuel pump system supplies fuel from the center tank or the inner wing tanks to the engines. The system has six main fuel pumps.

MAIN COMPONENTS TANK PUMPS
R	In normal operation each engine is supplied by one pump in the center tank or two pumps in its own side wing tank.
     All wing tank pumps remain on throughout the flight. They are fitted with pressure relief R	sequence valves which ensure that, when all pumps are running, the center tank pumps
will deliver fuel preferentially.

TRANSFER VALVES

Two electrical transfer valves are mounted in each wing to permit fuel transfer from outer to inner tank.

CROSS FEED VALVE

A cross feed valve controlled by a double motor allows both engines to be fed from one side or one engine to be fed from both sides.

ENGINE LP VALVES

The fuel flow to an engine can be stopped by its low pressure (LP) fuel valve, the closure of the LP fuel valve is by :
- the engine master switch, or
- the ENG FIRE PUSH pushbutton.

SUCTION VALVES

Closed by pumps pressure in normal operation, they allow engines to be fed by gravity if the inner tank pumps fail.

R	Note : Center tank pumps are not fitted with suction valves. Therefore, gravity feeding is
--	not possible from the center tank.





THY MSN 1856-2739 2928-2984 3010 3142-3185 3208-3257 3303-3341 3558-3603 3654 3718-3719
4629

- controls the transfer of fuel from the center tank to the wing tanks
- circulates fuel to cool the integrated drive generator (IDG).

I TANKS I

Tanks in the wings and the center section store fuel. There is a vent surge tank outboard of each wing tank.
When the aircraft has been refueled to maximum capacity, the fuel can expand by 2 % (20° temperature rise) without spilling.
R		There is an overpressure protector in each vent and wing tank, and between the center tank and the left wing tank.






0""'
0
"I'
0
0
I
0
.,
N
I
0
I
"u'
z

USABLE FUEL

WING TANKS
CENTER TANK
TOTAL

VOLUME
(liters)
2 X 7750
8200
23700

(US gallons)
2 X 2047
2166
6260

WEIGHT*
(KG)
2 X 6084
6437
18605

(LB)
2 X 13410
14190
41010

* Fuel density : 0.785 kg/I or 6.551 lb/US Gal.

THY MSN 0806-0810 2823-2868 2ggg 3015-3126 3207 3283 3350-3539 4643-4654


The main fuel pump system supplies fuel from the wing tanks to the engines. The system has four main fuel pumps.

MAIN COMPONENTS TANK PUMPS
In normal operation each engine is supplied from two pumps in its own side wing tank.

CROSS FEED VALVE

A cross feed valve controlled by a double motor allows both engines to be fed from one side or one engine to be fed from both sides.

ENGINE LP VALVES

The fuel flow to an engine can be stopped by its low pressure (LP) fuel valve. The closure of the LP fuel valve is controlled by :
- the engine master switch, or
- the ENG FIRE PUSH pushbutton.

SUCTION VALVES

Closed by pumps pressure in normal operation, they allow engines to be fed by gravity if the wing tank pumps fail.

- circulates fuel to cool the integrated drive generator (IDG)
- keeps fuel in the outer wing for wing bending and flutter relief.

I TANKS I

The fuel is stored in the wings and the center section. The wings have inner and outer tanks.
There is a vent surge tank outboard of the outer tank in each wing.
When the aircraft has been refueled to maximum capacity, the fuel can expand by 2 % (20° temperature rise) without spilling.
There is an overpressure protector in each vent outer and inner tank and between the center tank and the left inner tank.
INNER CENTER INNER TANK TANK TANK




0
"<'
I
0
0
I
0
.,
N
I
0
I
"u'

2

USABLE FUEL

OUTER TANKS
INNER TANKS
CENTER TANK
TOTAL

VOLUME
(liters)
880 X 2
7099 X 2
8250
24209

(US gallons)
232 X 2
1875 X 2
2180
6395

WEIGHT*
(KG)
691 X 2
5573 X 2
6476
19004

(LB)
1520 X 2
12286 X 2
14281
41893

* Fuel density : 0.785 kg/I or 6.551 lb/US Gal.


The main fuel pump system supplies fuel from the center tank or the inner wing tanks to the engines. The system has six main fuel pumps.

MAIN COMPONENTS TANK PUMPS
R	In normal operation each engine is supplied by one pump in the center tank or two pumps in its own side wing tank.
     All wing tank pumps remain on throughout the flight. They are fitted with pressure relief R	sequence valves which ensure that, when all pumps are running, the center tank pumps
will deliver fuel preferentially.

TRANSFER VALVES

Two electrical transfer valves are mounted in each wing to permit fuel transfer from outer to inner tank.

CROSS FEED VALVE

A cross feed valve controlled by a double motor allows both engines to be fed from one side or one engine to be fed from both sides.

ENGINE LP VALVES

The fuel flow to an engine can be stopped by its low pressure (LP) fuel valve, the closure of the LP fuel valve is by :
- the engine master switch, or
- the ENG FIRE PUSH pushbutton.

SUCTION VALVES

Closed by pumps pressure in normal operation, they allow engines to be fed by gravity if the inner tank pumps fail.

R	Note : Center tank pumps are not fitted with suction valves. Therefore, gravity feeding is
--	not possible from the center tank.

- Supplies fuel, in the correct quantities, to the fuel tanks during refueling.
- Supplies fuel to the engines and the Auxiliary Power Unit (APU).
- Controls the transfer of fuel from the center tank to the wing tanks.
- Circulates fuel to cool the Integrated Drive Generator (IDG).

I TANKS I

Fuel is stored in the wings, the center tank, and the ACT (Additional Center Tank), when installed.
There is a vent surge tank outboard of each wing tank.
When the aircraft has been refueled to maximum capacity, the fuel can expand by 2 % (20° temperature rise) without spilling.
There is an overpressure protector in each vent and wing tank, and between the center tank and the left wing tank.
<
<
0"_',
<
I
0
0
I
0
.,
N I

0
I
"<..'>
z

R










* Fuel density : 0.785 kg/I or 6.551 lb/US Gal.


The main fuel pump system supplies fuel from the wing tanks to the engines. The system has four main fuel pumps.

MAIN COMPONENTS TANK PUMPS
In normal operation each engine is supplied from two pumps in its own side wing tank.

CROSS FEED VALVE

A cross feed valve controlled by a double motor allows both engines to be fed from one side or one engine to be fed from both sides.

ENGINE LP VALVES

The fuel flow to an engine can be stopped by its low pressure (LP) fuel valve. The closure of the LP fuel valve is controlled by :
- the engine master switch, or
- the ENG FIRE PUSH pushbutton.

SUCTION VALVES

Closed by pumps pressure in normal operation, they allow engines to be fed by gravity if the wing tank pumps fail.

- Supplies fuel in the correct quantities to the fuel tanks during refueling.
- Supplies fuel to the engines and the Auxiliary Power Unit (APU).
- Controls the transfer of fuel from the center tank to the wing tanks.
- Circulates fuel to cool the Integrated Drive Generator (IDG).

I TANKS I

Fuel is stored in the wings, the center tank, and the ACTs (Additional Center Tanks), when installed.
There is a vent surge tank outboard of each wing tank.
When the aircraft has been refueled to maximum capacity, the fuel can expand by 2 % (20° temperature rise) without spilling.
There is an overpressure protector in each vent and wing tank, and between the center tank and the left wing tank.


_,
<
I
0
0
I
0
"N'
I

0
I
u
z

USABLE FUEL

WING TANKS
CENTER TANK

ACT
TOTAL WITH











NO ACT
1 ACT
2 ACTs

VOLUME
(liters)
2 X 7750
8200
2992
23700
26692
29684

(US gallons)
2 X 2047
2166
790
6260
7050
7840

WEIGHT*
(KG)
2 X 6084
6437
2349
18605
20953
23302

(LB)
2 X 13412
14191
5178
41015
46193
51371

* Fuel density : 0.785 kg/I or 6.551 lb/US Gal.


I ENGINE FEED I

GENERAL

The main fuel pump system supplies fuel from the wing tanks to the engines. The system has four main fuel pumps.

MAIN COMPONENTS TANK PUMPS
In normal operation each engine is supplied from two pumps in its own side wing tank.

CROSS FEED VALVE

A cross feed valve controlled by a double motor allows both engines to be fed from one side or one engine to be fed from both sides.

ENGINE LP VALVES

The fuel flow to an engine can be stopped by its low pressure (LP) fuel valve. The closure of the LP fuel valve is controlled by :
- the engine master switch, or
- the ENG FIRE PUSH pushbutton.

SUCTION VALVES

Closed by pumps pressure in normal operation, they allow engines to be fed by gravity if the wing tank pumps fail.

.......
"'<
.,
z
N
.,,
..N.....
"'
N
"N'
N
"'
...




NFC5-01-2810-003-A100AA









-











- - - -	-I	ClR 11(	x-FEED VALVE	Iftl11("-	- - -
I	,,,--.	I


mz
C')
mz
m"Tl m C
z


0
.0....
.N..
.,,

r oorn




11(

-,
I
I
XFR Cl)


LP
VALVE


LP
VALVE

,- -
I
I
Cl) XFR


OOTR 11<'
I
I

...
0
...
.N,,
...
0...

VALVES Cl)
I I I









DEFLR	VsEQLENCE

Cl) VALVES	:
I	I
I	I
i	I
IN-ETI
I	SEQUENCEV	I	VALVE1
I	VALVES	I REFLR	I

......
...
.,,
...


.
r;,

LII.E	VALVES

----- ..L

LJ
REFLR

--------LI

--I  VALVE
LJ	-
REFUEL

:I
---J

0...
...
.,,
.......
.......

P:--- COl.PLING

COl.PLING






r


The tanks empty in the following sequence :
1. center tank
2. inner tanks (down to 750 kg in each inner tank)
3. outer tanks (fuel transfered into the inner tanks)

CEN







E
PLJ,IP 1	MODE SEL	PUIII' 2	R 11< PUlll'S 2	L
:






0 OFF

ON

MAN


AUTO



AUTO STOP





"<'
0
0



-lt,t,IER CELL FULL

-  SLATS EXTENTED

< - NO ENG START
I

0  -21111 AFTER ENG START
I
0
;;;
N

I  -CTR 11< LO LVL
."'-.'.'	sm

* Each center tank pump stops until approximately 500 kg (1100 lb) of its associated inner tank fuel has been used (when the fuel level reaches the underfull sensors).



THY MSN 1856-2739 2928-2984 3010 3142-3185 3208-3257 3303-3341 3558-3603 3654 3718-3719
3896-4629

.3',
z
0
.0 .
.0N.
.N.
N

...
0.,,
...
N
...
0....
...
...
.,,
0
...,.,
.....


NFC5-01-2810-003-A200AA













r

mz
C')
mz
m"Tl m C
z














I

r;,



REFUEL


REFUEL








r

1. center tank (fuel transferred into wing tanks)
2. wing tanks

CENTER TANK FUEL TRANSFER

The transfer system controls the flow of fuel from the center tank to the wing tanks. The Fuel Level Sensing Control Unit (FLSCU) has automatic control of the transfer valve. When the transfer valve is open, fuel from the wing tank pumps flows through the jet pump and creates a suction in it. This suction moves the fuel from the center tank to the related wing tank.

Center tank transfer valve control logic

APU	X FEED	ENG 2
F
u


ICTRTK

ON	E
CTR TK	L

L XFR	MODE SEL	R XFR FAULT
6	rD





0 OFF



MAN



 OPENING	

ON	0

AUTO	CTR TK
L XFR VALVE



<
<
0
0
N
<
I
0
0
.6, -WING TAN< FULL-------TI-ME----
N	DELAY
I
o -ClR 1K LO LVL,	,
I
."u..'

R The FLSCU automatically closes the associated center tank transfer valve when the wing tank is full. The FLSCU then reopens the center tank transfer valve when the engine has used 250 kg (550 lb) of wing tank fuel (when the fuel level reaches the underfull sensors).

.3',
.z..
.......
...
.......
...
.......


NFC5-01-2810-003-A300AA










r













-  1""-  -  -  -  -  -  -	-  -	•-  -
- - - -	WING 1l<I CTR Tl(	x-FEEo VALVE	i WING
I	,,.---.	I

mz
C')
mz
m"Tl m C
z







l REFUEL/
l DEFUEL
LINE


LJ







ACT
XFR PLMP ---++-	LJ

r;,

REFUEL

REFUEL

[?/  COUPLING	COUPLING




I	[
PRESSURE	II-,	m; .

1. additional center tank (fuel transferred into the center tank )
2. center tank (fuel transferred into wing tanks)
3. wing tanks

CENTER TANK FUEL TRANSFER

The transfer system controls the flow of fuel from the center tank to the wing tanks. The Fuel Level Sensing Control Unit (FLSCU) has automatic control of the transfer valve. When the transfer valve is open, fuel from the wing tank pumps flows through the jet pump and creates a suction in it. This suction moves the fuel from the center tank to the related wing tank.

Center tank transfer valve control logic










0			OPENING	
0 I	ON
"N '	Al/TO  TIME	CTR 11<
p I	-	WING TAN< FULL	L XFR VALVE
0

I	-CTR 1K LO LVL
u
z





5ll'J


R The FLSCU automatically closes the associated center tank transfer valve when the wing R tank is full. The FLSCU then reopens the center tank transfer valve when the engines have R  used 250 kg (550 lb) of wing tank fuel (when the fuel level reaches the underfull sensors).

<
.3',
z


':'
.N..

NFC5-01-2810-003-A303AA	mz
C')
mz
m"Tl m C




r













APU LP  I
VALVE	I


ACT VENT VALVE
Af'BIENT + VENT 	 CABIN AIR 	
PRESSURE
AIR SHUT	ACT
OFF VALVE


FUEL FEED  SEQUENCE

The wing tanks feed fuel to the engines. The tanks empty in the following sequence :
1. additional center tank (fuel transferred into the center tank )
2. center tank (fuel transferred into wing tanks)
3. wing tanks

CENTER TANK FUEL TRANSFER

The transfer system controls the flow of fuel from the center tank to the wing tanks. The Fuel Level Sensing Control Unit (FLSCU) has automatic control of the transfer valve. When the transfer valve is open, fuel from the wing tank pumps flows through the jet pump and creates a suction in it. This suction moves the fuel from the center tank to the related wing tank.

Center tank transfer valve control logic










0			OPENING	
0 I	ON
"N '	Al/TO  TIME	CTR 11<
p I	-	WING TAN< FULL	L XFR VALVE
0

I	-CTR 1K LO LVL
u
z





5ll'J


R The FLSCU automatically closes the associated center tank transfer valve when the wing R tank is full. The FLSCU then reopens the center tank transfer valve when the engines have R  used 250 kg (550 lb) of wing tank fuel (when the fuel level reaches the underfull sensors).


R	 With the MODE SEL in MAN position, the center tank pumps will run. In manual mode the R	CTR TK PUMP pushbuttons must be selected off when the center tank is empty.

FUEL TRANSFER FROM OUTER TO INNER TANKS

The transfer valves automatically open when the inner tank fuel reaches the low level (about 750 kg/1650 lb), thus permitting fuel to drain from the outer to inner tanks.
When open, the valves are latched open. They will automatically close at the next refuel operation (MODE SELECT at REFUEL position).

Note: 1. Two level sensors are installed in each inner tank. Each sensor controls two transfer valves, one in each wing, ensuring that transfer is simultaneous in both wings.
2. The 750 kg/1650 lb value is based on a level aircraft attitude with no acceleration. During steep descents or accelerations/decelerations, the transfer valves may open with more than 750 kg/1650 lb of fuel in each inner tank and the low level warning may be triggered.

ECAM INDICATION










.,
<
0e
<
I
0"'
0
eI
"N'
0
I
"u '
z





TRANSFER FROM OUTER TO INNER TAtKS
+
INNER TAN< FEEDING


A special fuel pump supplies fuel for APU startup when fuel feed pressure is low (due to loss of tank pumps or loss of normal AC electrical supply). This pump normally runs off the AC ESS SHED, but runs off the AC STAT INV BUS if the AC ESS SHED fails.

I FUEL RECIRCULATION  SYSTEM I

See also 1.70.45 "IDG COOLING SYSTEM".
Some of the fuel supplied to each engine goes from the high-pressure fuel line in that engine, through the integrated drive generator (IDG) heat exchanger (where it absorbs heat), to the fuel return valve, and to the outer fuel tank.
This operation ensures the IDG cooling when the oil temperature is high or when at low engine power.
The FADEC controls the fuel return valve.
If the outer tank is already full, the fuel overflows to the inner tank through a spill pipe. If center tank is feeding, the wing tank will tend to overfill and the system automatically selects the CTR TK PUMP off when the inner tank is full. The wing tank pumps will feed until the engine have used approximately 500 kg (1100 lb) of fuel when the fuel level reaches the underfull sensors. The logic circuits then restart the center tank pumps.
























<
<
0
0
N
<
I
0"'
0
0I

N I

With the MODE SEL in MAN position, the center tank transfer valves will open.
Wing tank overflow must be prevented by selecting the CTR TK XFR phushbuttons to off when the wing tanks are full. They must also be selected off when the center tank is empty.
ECAM INDICATION






0
I

."u..'

TRANSFER FROM CENTER TAN<


A special fuel pump supplies fuel for APU startup when fuel feed pressure is low (due to loss of tank pumps or loss of normal AC electrical supply). This pump normally runs off the AC ESS SHED, but runs off the AC STAT INV BUS if the AC ESS SHED fails.

I FUEL RECIRCULATION  SYSTEM I

See also 1.70.45 "IDG COOLING SYSTEM".
Some of the fuel supplied to each engine goes from the high-pressure fuel line in that engine, through the integrated drive generator (IDG) heat exchanger (where it absorbs heat), to the fuel return valve, and back to the wing tank.
This operation ensures the IDG cooling when the oil temperature is high or when at low engine power.
The FADEC controls the fuel return valve.


With MODE SEL in the MAN position, the center tank pumps will run. In manual mode, the CTR TK PUMP pushbuttons must be selected off, when the center tank is empty.

FUEL TRANSFER FROM OUTER TO INNER TANKS

The transfer valves automatically open, when the inner tank fuel reaches the low level (about 750 kg/1650 lb), thus enabling the fuel to drain from the outer to inner tanks.
When open, the valves are latched open. They will automatically close at the next refueling operation.

Note: 1. Two level sensors are installed in each inner tank. Each sensor controls two transfer valves, one in each wing, ensuring simultaneous transfer to both wings.
2. The 750 kg/1650 lb value is based on a level aircraft attitude, with no acceleration. During steep descent or acceleration/deceleration, the transfer valves may open with more than 750 kg/1650 lb of fuel in each inner tank, and the low level warning may be triggered.

ECAM INDICATION





<
<
0
N N
<
'
0
0
0'
"N '
'





TRANSFER FROM OUTER TO INNER TANKS
+
INNER TANK FEEDING


A special fuel pump supplies fuel for APU startup when fuel feed pressure is low (due to loss of tank pumps or loss of normal AC electrical supply). This pump normally runs off the AC ESS SHED, but runs off the AC STAT INV BUS if the AC ESS SHED fails.

I FUEL RECIRCULATION  SYSTEM I

See also 1.70.40 "IDG COOLING SYSTEM".
Some of the fuel supplied to each engine goes from the high-pressure fuel line in that engine, through the integrated drive generator (IDG) heat exchanger (where it absorbs heat), to the fuel return valve, and to the outer fuel tank.
This operation ensures the IDG cooling when the oil temperature is high or when at low engine power.
The FADEC controls the fuel return valve.
If the outer tank is already full, the fuel overflows to the inner tank through a spill pipe. If center tankis feeding, the wing tank will tend to overfill and the system automatically selects the CTR TK PUMP off when the inner tank is full. The wing tank pumps will feed until the engine have used approximately 500 kg (1100 lb) of fuel when the fuel level reaches the underfull sensors. The logic circuits then restart the center tank pumps.
FUEL TAN<







HP PUMP



""'' "I'
0"'
0


TO BURNERS

I	FUEL RETURN
N	VALVE

0
I

."u..'.
z

TO OUTER
TAN<


With MODE SEL in the MAN position, the center tank pumps will run. In manual mode, the CTR TK PUMP pushbuttons must be selected off, when the center tank is empty.

FUEL TRANSFER FROM OUTER TO INNER TANKS

The transfer valves automatically open, when the inner tank fuel reaches the low level (about 750 kg/1650 lb), thus enabling the fuel to drain from the outer to inner tanks.
When open, the valves are latched open. They will automatically close at the next refueling operation.

Note: 1. Two level sensors are installed in each inner tank. Each sensor controls two transfer valves, one in each wing, ensuring simultaneous transfer to both wings.
2. The 750 kg/1650 lb value is based on a level aircraft attitude, with no acceleration. During steep descent or acceleration/deceleration, the transfer valves may open with more than 750 kg/1650 lb of fuel in each inner tank, and the low level warning may be triggered.

ECAM INDICATION





<
<
0
N N
<
'
0
0
0'
"N '
'





TRANSFER FROM OUTER TO INNER TANKS
+
INNER TANK FEEDING










THY MSN 2609-2739 2928-2984 3010 3142-3185 3208-3257 3303-3341 3558-3603 3654 3718-3719
4629


A special fuel pump supplies fuel for APU startup when fuel feed pressure is low (due to loss of tank pumps or loss of normal AC electrical supply). This pump normally runs off the AC ESS SHED, but runs off the AC STAT INV BUS if the AC ESS SHED fails.

I FUEL RECIRCULATION  SYSTEM I

See also 1.70.45 "IDG COOLING SYSTEM".
Some of the fuel supplied to each engine goes from the high-pressure fuel line in that engine, through the integrated drive generator (IDG) heat exchanger (where it absorbs heat), to the fuel return valve, and to the outer fuel tank.
This operation ensures the IDG cooling when the oil temperature is high or when at low engine power.
The FADEC controls the fuel return valve.
If the outer tank is already full, the fuel overflows to the inner tank through a spill pipe. If center tank is feeding, the wing tank will tend to overfill and the system automatically selects the CTR TK PUMP off when the inner tank is full. The wing tank pumps will feed until the engine have used approximately 500 kg (1100 lb) of fuel when the fuel level reaches the underfull sensors. The logic circuits then restart the center tank pumps.

























THY MSN 2609-2739 2928-2984 3010 3142-3185 3208-3257 3303-3341 3558-3603 3654 3718-3719
4629


R	With the MODE SEL in MAN position, the center tank transfer valves will open.
R	 Wing tank overflow must be prevented by selecting the CTR TK XFR pushbuttons off when R	the wing tanks are full. They must also be selected off when the center tank is empty.

ACT TO CENTER TANK TRANSFER

Automatic control of the transfer occurs after takeoff at slats retraction : It is initiated, provided the center tank high level sensor has been dry for ten minutes and the ACT is not empty.
The fuel transfer from the ACT to the center tank is made by pressurization of the tank by closing the ACT vent valve and opening the air shut off valve. When the ACT is empty, the ACT transfer valve, the air shut off valve and inlet valve shut and the ACT vent valve opens.

Note : If during the transfer, the center tank high level becomes wet, transfer from the ACT
--	stops by closing the ACT transfer valve. The transfer valve reopens when the center tank high level sensor becomes dry for ten minutes.

Selecting the ACT pushbutton to FWD will open the ACT transfer valve, the ACT inlet valve and start the ACT transfer pump. It must be returned to AUTO when the ACT is empty. Center tank overflow must be prevented by returning the ACT pushbutton to AUTO when the center tank is full.
	ECAM  INDICATION	









<
<
0
0
"<'
I
0"'
0
I
0

"N'
I


ACT QUANTITY	ACT TO CENTER TAN<

I	XFR S'mlOL

"u '
z

TRANSFER FROM CENTER TAl'I< AND FROM ACT


A special fuel pump supplies fuel for APU startup when fuel feed pressure is low (due to loss of tank pumps or loss of normal AC electrical supply). This pump normally runs off the AC ESS SHED, but runs off the AC STAT INV BUS if the AC ESS SHED fails.

I FUEL RECIRCULATION  SYSTEM I

See also 1.70.40 "IDG COOLING SYSTEM".
Some of the fuel supplied to each engine goes from the high-pressure fuel line in that engine, through the integrated drive generator (IDG) heat exchanger (where it absorbs heat), to the fuel return valve, and back to the wing tank.
This operation ensures the IDG cooling when the oil temperature is high or when at low engine power.
The FADEC controls the fuel return valve.
FUEL TAN<










""'' "I'
0"'
0
I
.,
N
I



TO BURNERS


FUEL RETURN VALVE

0
I

"u'
z

TO WING
TAN<


With the MODE SEL in MAN position, the center tank transfer valves will open.
Wing tank overflow must be prevented by selecting the CTR TK XFR pushbuttons off when the wing tanks are full. They must also be selected off when the center tank is empty.
ECAM INDICATION








<
<
"'
"<'
I
0"'
0
I
0
"N '
I
0
I

"u '
z

TRANSFER FROM CENTER TANK


A special fuel pump supplies fuel for APU startup when fuel feed pressure is low (due to loss of tank pumps or loss of normal AC electrical supply). This pump normally runs off the AC ESS SHED, but runs off the AC STAT INV BUS if the AC ESS SHED fails.

I FUEL RECIRCULATION  SYSTEM I

See also 1.70.45 "IDG COOLING SYSTEM".
Some of the fuel supplied to each engine goes from the high-pressure fuel line in that engine, through the integrated drive generator (IDG) heat exchanger (where it absorbs heat), to the fuel return valve, and back to the wing tank.
This operation ensures the IDG cooling when the oil temperature is high or when at low engine power.
The FADEC controls the fuel return valve.


With the MODE SEL in the MAN position, the center tank transfer valves will open. Wing tank overflow must be prevented by setting the CTR TK XFR pushbuttons off, when the wing tanks are full. They must also be selected off when the center tank is empty.

ACT 1 + 2 /41 TO CENTER TANK TRANSFER
Automatic control of the transfer occurs after takeoff at slats' retraction : It is initiated, provided the center tank high level sensor has been dry for ten minutes and either ACT is not empty.
The fuel transfer from the ACTs to the center tank is made by pressurizing the tank, by closing the ACT vent valves and opening the air shut-off valve.
ACT2 transfers first via the ACT transfer valve and ACT2 inlet valve.
When ACT2 is empty, the ACT2 inlet valve closes, and the ACT1 inlet valve opens. When both ACTs are empty, the ACT transfer valve, the air shutoff valve and both ACT inlet valves shut, and both ACT vent valves open.

Note : If, during the transfer, the center tank high level becomes wet, transfer from the ACT
--	stops by closing the ACT transfer valve. The transfer valve reopens when the center tank high level sensor becomes dry for ten minutes.

Selecting the ACT pushbutton to FWD opens the ACT transfer valve and starts the ACT transfer pump. ACT2 empties first, then ACT1. It must be returned to AUTO, when the ACT is empty.
Center tank overflow must be prevented by returning the ACT pushbutton to AUTO, when the center tank is full.
ECAM INDICATION

FUEL





6
o::::J


"<C
I
0"'
0
I

ACT QUANTITY ----i-i,;; =;;;;;;;;;; •=.-r	''
17 KG


--- -

.,
N
I
0
I
"u'
z


ACT TO CENTER TAIII< XFR SYMBOL
TRANSFER FROM ACT



THY MSN 3637 3673-3688 3738


A special fuel pump supplies fuel for APU startup when fuel feed pressure is low (due to loss of tank pumps or loss of normal AC electrical supply). This pump normally runs off the AC ESS SHED, but runs off the AC STAT INV BUS if the AC ESS SHED fails.

I FUEL RECIRCULATION  SYSTEM I

See also 1.70.45 "IDG COOLING SYSTEM".
Some of the fuel supplied to each engine goes from the high-pressure fuel line in that engine, through the integrated drive generator (IDG) heat exchanger (where it absorbs heat), to the fuel return valve, and back to the wing tank.
This operation ensures the IDG cooling when the oil temperature is high or when at low engine power.
The FADEC controls the fuel return valve.











<
<
'()
0
<
r--
0
0
0'
"N'
p
0
."u.'.'
- 
One (two) refueling point(s) is (are) installed under the wings allowing the refueling from either the right or left ( ) side of the aircraft.
- A refuel panel is located on the fuselage side beneath the right wing or under the right or left wing adjacent to the refuel coupling.


I,>		REFUEL P REFUEL COUP


A gallery connects the refuel coupling to the refuel valve of each tank.
Refueling is normally automatic, the required fuel load being set on the preselector. Manual control is also available.
Automatic refueling starts by the outer cells. If the selected fuel load exceeds the wing tank capacity, the center tank is refuelled simultaneously.
When an outer cell is full the fuel overflows into the inner cell through a spill pipe. Refuel valves close automatically when the tanks contain the preselected load or when sensors detect a high fuel level.
The aircraft can be refueled when only battery power is available.
The wing tanks can be refueled by gravity through refueling points on top of the wings. A transfer valve between the engine feed system and the refueling gallery permits :
- tank pumps to transfer fuel from one tank to another.
- defueling through the refuel coupling. Approximate refueling time at nominal pressure is :
- 17 minutes for wing tanks.
- 20 minutes for all tanks.




<
<
'()
0
Ill
r--
0
0
0 '
"N '
'
            
SUCTION VALVE REFUEL COUPLING







TRANSFER VALVE

---+-1- HI LEVEL
SENSOR



REFUEL
VALVE

0
."u.'.'
     
REFUELING GALLERY HI LEVEL SENSOR




THY MSN 1856-2739 2928-2984 3010 3142-3185 3208-3257 3303-3341 3558-3603 3654 3718-3719
3896-4629


The FOi is a computerized system that :
- transmits the actual total fuel mass, as well as the quantity and temperature of fuel in the tanks, to the ECAM.
- controls automatic refueling.
Two channels perform fuel computations : channel 2 activates automatically if channel 1 fails.
The FOi system comprises :
- an FQI computer.
- a set of capacitance probes in each tank to measure fuel level and temperature.
- one densitometer (cadensicon) sensor in each wing inner tank permitting the calculation of the fuel quantity.
- one Capacitance Index Compensator (CIC) in each inner tank giving the dielectric constant of the fuel in case of cadensicon failure.
- a quantity indicator for each tank installed on the refuel/defuel panel.
- a preselector on the refuel/defuel panel that shows the preselected and actual total fuel quantity.

FUEL LEVEL  SENSING CONTROL UNIT (FLSCUl

The fuel level system generates fuel-level and fuel-temperature signals in order to operate the appropriate switching functions for refueling and defueling and control the IDG cooling recirculation system and the center-tank-to-wing-tank fuel transfer system.
The FLSCU comprises :
- fuel level sensors in the tanks to sense high, low, and overflow levels.
- a fuel temperature sensor to control the IDG cooling recirculation.
When fuel quantity in one wing tank goes below 750 kg (1650 lb), the low-level sensor triggers the LO LVL warning on ECAM.














THY MSN 1856-2739 2928-2984 3010 3142-3185 3208-3257 3303-3341 3558-3603 3654 3718-3719
3896-4629


R	One (two) refueling point(s) is (are) installed under the wings allowing the refueling from R	 either the right or left ( ) side of the aircraft.
R	- A refuel panel is located either on the fuselage side beneath the right wing or under the R		right wing adjacent to the refuel coupling.
R



"<'
0
0
N
<
I
0
0
 
REFUEL P REFUEL COU

 I 0., N
I
p
0
I
."u..'

A gallery connects the refuel coupling to the refuel valve of each tank.
Refueling is normally automatic, the required fuel load being set on the preselector. Manual control is also available.
Automatic refueling starts with the wing tanks. If the preselected fuel load exceeds the capacity of the wing tanks, the center tank receives fuel simultaneously.
Refuel valves close automatically when the tanks contain the preselected load or when sensors detect a high fuel level.
The aircraft can be refueled when only battery power is available.
The wing tanks can be refueled by gravity through refueling points on top of the wings. A transfer valve between the engine feed system and the refueling gallery permits :
- tank pumps to transfer fuel from one tank to another.
- defueling through the refuel coupling. Approximate refueling time at nominal pressure is :
- 17 minutes for wing tanks.
- 20 minutes for all tanks.
CENTER TAN<	HI LEVEL
-+-+---SENSOR

SUCTION VALVE
g<	REFUEL COUPLING
!	REFUEL VALVE----------...,:
0
0

,'---TRANSFER
VALVE

I
.,
N
I	X FEED VALVE
I	REFUELING GALLERY

	REFUEL
VALVE
TRANSFER /DEFUEL VALVE

."u..'

HI LEVEL SENSOR


THY MSN 0806-0810 2823-2868 2ggg 3015-3126 3207 3283 3350-3539 4643-4654


The FOi is a computerized system that :
- transmits the actual total fuel mass, as well as the quantity and temperature of fuel in the tanks, to the ECAM.
- controls automatic refueling.
Two channels perform fuel computations : channel 2 activates automatically if channel 1 fails.
The FOi system has :
- an FQI computer.
R	- a set of capacitance probes in each tank to measure fuel level. R	- sensors in each wing tank to measure fuel temperature.
R	- devices for measuring fuel characteristics such as density
R	- a multi tank indicator allowing the quantity for each tank to be displayed on the R		refuel/defuel panel.

FUEL LEVEL  SENSING CONTROL UNIT (FLSCUl

The fuel level system generates fuel-level and fuel-temperature signals in order to operate the appropriate switching functions for refueling and defueling and control the IDG cooling recirculation system and the center-tank-to-wing-tank fuel transfer system and ACT ( ) to center tank fuel transfer system.
The FLSCU comprises :
- fuel level sensors in the tanks to sense high, low, and overflow levels.
- a fuel temperature sensor to control the IDG cooling recirculation.
When fuel quantity in one wing tank goes below 750 kg (1650 lb), the low-level sensor triggers the LO LVL warning on ECAM.











<
<
0"'
"<'
I
r--

One (two) refueling point(s) is (are) installed under the wings allowing the refueling from either the right or left ( ) side of the aircraft.
- A refuel panel is located either under the right or left wing, adjacent to the refuel coupling or on the fuselage side beneath the right wing.



REFUEL

0
0
I
0
"N'
I

0
I
"u'
2

A gallery connects the refuel coupling to the refuel valve of each tank. Refueling is normally automatic, the required fuel load being set on the preselector. Manual control is also available. Automatic refueling starts with the wing tanks. If the preselected fuel load exceeds the capacity of the wing tanks, the center tank receives fuel simultaneously. Refuel valves close automatically when the tanks contain the preselected load or when sensors detect a high fuel level. The aircraft can be refueled when only battery power is available. The wing tanks can be refueled by gravity through refueling points on top of the wings.
A transfer valve between the engine feed system and the refueling gallery permits :
- tank pumps to transfer fuel from one tank to another.
- defueling through the refuel coupling. Approximate refueling time at nominal pressure is :
- 17 minutes for wing tanks.
- 27 minutes for all tanks.

       FUEL PUMPS SUCTION VALVE

CENTER TANK	HI LEVEL
+-+---SENSOR

REFUEL COUPLING'----._
;	REFUEL VALVE
0
""''
r--




ED VALVE

" , 

TRANSFER
VALVE

ACT REFUEL VE

0
0
I
0
"N '
I
0
I
"u'
2
 
TRANSFER VALVE REF REFUELING GALLERY VAL
HI LEVEL SENSOR
INLET VALVES

ACT 1
ACT 2


The FOi is a computerized system that :
- transmits the actual total fuel mass, as well as the quantity and temperature of fuel in the tanks, to the ECAM.
- controls automatic refueling.
Two channels perform fuel computations : channel 2 activates automatically if channel 1 fails.
The FOi system has :
- an FQI computer.
R	- a set of capacitance probes in each tank to measure fuel level. R	- sensors in each wing tank to measure fuel temperature.
R	- devices for measuring fuel characteristics such as density
R	- a multi tank indicator allowing the quantity for each tank to be displayed on the R		refuel/defuel panel.

FUEL LEVEL  SENSING CONTROL UNIT (FLSCUl

The fuel level system generates fuel-level and fuel-temperature signals in order to operate the appropriate switching functions for refueling and defueling and control the IDG cooling recirculation system and the center-tank-to-wing-tank fuel transfer system and ACT ( ) to center tank fuel transfer system.
The FLSCU comprises :
- fuel level sensors in the tanks to sense high, low, and overflow levels.
- a fuel temperature sensor to control the IDG cooling recirculation.
When fuel quantity in one wing tank goes below 750 kg (1650 lb), the low-level sensor triggers the LO LVL warning on ECAM.


One (two) refueling point(s) is (are) installed under the wings allowing the refueling from either the right or left ( ) side of the aircraft.
- A refuel panel is located either under the right or left wing, adjacent to the refuel coupling or on the fuselage side beneath the right wing.


<
<
N
"<'
,-I.
0
0
 I 0., N
I





-----REFUEL  PANEL
REFUEL COUPLING

0
I

."u..'

REFUEL PANEL


A gallery connects the refuel coupling to the refuel valve of each tank.
Refueling is normally automatic, the required fuel load being set on the preselector. Manual control is also available.
Automatic refueling starts with the wing tanks. If the preselected fuel load exceeds the capacity of the wing tanks, the center tank receives fuel simultaneously.
Refuel valves close automatically when the tanks contain the preselected load or when sensors detect a high fuel level.
The aircraft can be refueled when only battery power is available.
The wing tanks can be refueled by gravity through refueling points on top of the wings. A transfer valve between the engine feed system and the refueling gallery permits :
- tank pumps to transfer fuel from one tank to another.
- defueling through the refuel coupling. Approximate refueling time at nominal pressure is :
- 17 minutes for wing tanks.
R	- 25 minutes for all tanks.





,-I'.
0
0
I
0.,
N
       
FUEL PIJVIPS SUCTION VALVE
REFUEL COUPLING'	--.
REFUEL VALVE

CENTER TAN< -+-+---SENHSIOLERVEL
r	 TRANSFER
VALVE
ACT REFUEL VALVE
REFUEL
VALVE
X FEED VALVE	TRANSFER /DEFUEL

I	REFUELING GALLERY *U VALVE
0	SENSOR	--- _	ACT INLET
I	VALVE
."u..'


The FOi is a computerized system that :
- transmits the actual total fuel mass, as well as the quantity and temperature of fuel in the tanks, to the ECAM.
- controls automatic refueling.
Two channels perform fuel computations : channel 2 activates automatically if channel 1 fails.
The FOi system has :
- an FQI computer.
R	- a set of capacitance probes in each tank to measure fuel level. R	- sensors in each wing tank to measure fuel temperature.
R	- devices for measuring fuel characteristics such as density
R	- a multi tank indicator allowing the quantity for each tank to be displayed on the R		refuel/defuel panel.

FUEL LEVEL  SENSING CONTROL UNIT (FLSCUl

The fuel level system generates fuel-level and fuel-temperature signals in order to operate the appropriate switching functions for refueling and defueling and control the IDG cooling recirculation system and the center-tank-to-wing-tank fuel transfer system and ACT ( ) to center tank fuel transfer system.
The FLSCU comprises :
- fuel level sensors in the tanks to sense high, low, and overflow levels.
- a fuel temperature sensor to control the IDG cooling recirculation.
When fuel quantity in one wing tank goes below 750 kg (1650 lb), the low-level sensor triggers the LO LVL warning on ECAM.

A®318/A319/A320/A321
TURKISH AIRLINES

FLIGHT CREW OPERATING MANUAL
FUEL
DESCRIPTION
1.28.10	p 9


SEQ 001 I REV 24
FUEL SYSTEM ARCHITECTURE
R
































m
<
0
0
I
0,
0
0
I
0
"N'
I
0
I
."u..'





<
<
0

<

0
0
I
.N ,
N I
p
0
I
."u..'

(D L (R) TK PUMPS 1(2) pb sw


On

OFF
FAULT light

Pump is on but fuel feeds only when center tank pumps delivery pressure drops below threshold.
Pump is OFF and OFF button lights up white.
Amber light comes on, and ECAM caution comes on, when the delivery pressure drops. It does not come on when OFF is selected.

(D MODE SEL pb sw
AUTO	Control of center tank pumps is automatic.
• They run at engine start for 2 minutes.
• Before or after engine start sequence, the pumps run if the slats are retracted.
             · They stop automatically 5 minutes after center tank low level is reached.
MAN		Flight crew controls the center tank pumps manually with center tank pumps pushbutton switches.
FAULT light Amber light comes on, and ECAM caution comes on when center tank has more than 250 kg (550 lb) of fuel and the left or right wing tank has less than 5000 kg (11000 lb).
Q) CTR  TK PUMP 1(2) pb sw


On

OFF
FAULT light
• 
Pump runs if MAN mode is selected on MODE SEL pushbutton switch.
• Pump is automatically controlled when AUTO mode is selected. Pump is OFF and OFF button lights up white.
Amber light comes on, associated with ECAM caution, when the pump is in operation and the delivery pressure drops.






THY MSN 1856-2739 2928-2984 3010 3142-3185 3208-3257 3303-3341 3558-3603 3654 3718-3719
3896-4629










































THY MSN 1856-2739 2928-2984 3010 3142-3185 3208-3257 3303-3341 3558-3603 3654 3718-3719
3896-4629





<
<
0"'
<

0
0
I
.N ,
N I
p
0
I
."u..'

(D L (R) TK PUMPS pb sw


On OFF
FAULT light

Pump is on.
Pump is OFF and OFF button lights up white.
Amber light comes on, and ECAM caution comes on, when the delivery pressure drops. It does not come on when OFF is selected.

(D MODE SEL pb sw
AUTO	Control of center tank transfer valves is automatic.
- They open if the associated wing tank is not full.
- They close automatically five minutes after the center tank reaches its low level.
MAN	Flight crew controls the center tank transfer valves manually with CTR TK XFR pushbutton switches.
FAULT light	Amber light comes on, and ECAM caution comes on, when :
- The center tank has more than 250 kg (550 lb) of fuel and the left or right wing tank has less than 5000 kg (11000 lb) and
- the MODE SEL pushbutton switch is at AUTO.
0 CTR  TK UR) XFR  pb SW


On



OFF
FAULT light


The transfer valve opens if MAN mode is selected with the MODE SEL pushbutton switch.
- The transfer valve is controlled automatically when the flight crew selects AUTO mode.
The transfer valve is closed.
Amber light comes on, as does the ECAM caution, if the associated wing tank overflows.







<
<
0"'
<

0
0
I
.N ,
N I
p
0
I
."u..'

(D L (R) TK PUMPS pb sw


On OFF
FAULT light

Pump is on.
Pump is OFF and OFF button lights up white.
Amber light comes on, and ECAM caution comes on, when the delivery pressure drops. It does not come on when OFF is selected.

(D MODE SEL pb sw
AUTO	Control of center tank transfer valves is automatic.
- They open if the associated wing tank is not full.
- They close automatically five minutes after the center tank reaches its low level.
MAN	Flight crew controls the center tank transfer valves manually with CTR TK XFR pushbutton switches.
FAULT light	Amber light comes on, and ECAM caution comes on, when :
- The center tank has more than 250 kg (550 lb) of fuel and the left or right wing tank has less than 5000 kg (11000 lb) and
- the MODE SEL pushbutton switch is at AUTO.
0 CTR  TK UR) XFR  pb SW


On



OFF
FAULT light


The transfer valve opens if MAN mode is selected with the MODE SEL pushbutton switch.
- The transfer valve is controlled automatically when the flight crew selects AUTO mode.
The transfer valve is closed.
Amber light comes on, as does the ECAM caution, if the associated wing tank overflows.


Off	The valve closes, and button does not light up.
ON	: The valve opens, and the ON button comes on white. OPEN light : This green light comes on, when the valve is fully open.
® ACT pushbutton
AUTO	Control of the ACTs /41 transfer is automatic.
- The automatic forward transfer occurs, if :
• The aircraft is in flight, and
· The slats are retracted, and
• At least one ACT low level sensor is wet, and
• The center tank high level sensor has been dry for at least 10 minutes.
- The automatic forward transfer stops, as soon as one of the above conditions is not met.
FWD	Manual transfer to the center tank is initiated by opening :
- The ACT transfer valve
- The ACT 1 or 2 /41 inlet valve.
The ACT transfer pump is then commanded on.
FAULT light	Amber light and associated ECAM caution come on, when :
- The center tank has less than 3000 kg (6614 lb) of fuel, and one ACT
/41 has more than 250 kg (550 lb) of fuel, and
- The ACT pushbutton is on AUTO.

R	Note : When no ACT is installed, the pushbutton is inoperative.




(D L (R) TK PUMPS pb sw


On OFF
FAULT light

Pump is on.
Pump is OFF and OFF button lights up white.
Amber light comes on, and ECAM caution comes on, when the delivery pressure drops. It does not come on when OFF is selected.

(D MODE SEL pb sw
AUTO	Control of center tank transfer valves is automatic.
- They open if the associated wing tank is not full.
- They close automatically five minutes after the center tank reaches its low level.
MAN	Flight crew controls the center tank transfer valves manually with CTR TK XFR pushbutton switches.
FAULT light	Amber light comes on, and ECAM caution comes on, when :
- The center tank has more than 250 kg (550 lb) of fuel and the left or right wing tank has less than 5000 kg (11000 lb) and
- the MODE SEL pushbutton switch is at AUTO.
0 CTR  TK UR) XFR  pb SW


On



OFF
FAULT light


The transfer valve opens if MAN mode is selected with the MODE SEL pushbutton switch.
- The transfer valve is controlled automatically when the flight crew selects AUTO mode.
The transfer valve is closed.
Amber light comes on, as does the ECAM caution, if the associated wing tank overflows.


Off	The valve closes, and button does not light up.
ON	: The valve opens, and the ON button comes on white. OPEN light : This green light comes on, when the valve is fully open.
® ACT pushbutton
AUTO	Control of the ACTs /41 transfer is automatic.
- The automatic forward transfer occurs, if :
• The aircraft is in flight, and
· The slats are retracted, and
• At least one ACT low level sensor is wet, and
• The center tank high level sensor has been dry for at least 10 minutes.
- The automatic forward transfer stops, as soon as one of the above conditions is not met.
FWD	Manual transfer to the center tank is initiated by opening :
- The ACT transfer valve
- The ACT 1 or 2 /41 inlet valve.
The ACT transfer pump is then commanded on.
FAULT light	Amber light and associated ECAM caution come on, when :
- The center tank has less than 3000 kg (6614 lb) of fuel, and one ACT
/41 has more than 250 kg (550 lb) of fuel, and
- The ACT pushbutton is on AUTO.

R	Note : When no ACT is installed, the pushbutton is inoperative.






,------- HI.	LVL.--
LEFT	CTR	RIGH_ T .._,

	.0

Cg	Cg
NORM	NORM	NORM +	0
SHUT	SHUT	SHUT
r-	MODE SELECT ----.._ TEST	,-------,	;®
I

""''

0,---_;  ---I??@: 			®



0
"I'

DEFUEL  XFR  _/  OPE,N 	LTS	NORM	I	0 

0"'
0

PRE-S,E,L-,EC,T-E,D	REFUEL	ACTUAL

I	,--,..	0

0I	KG X 1000	,-, ,-

N
"N'
I
I



f-----,, -...?..

00T	' ::==8=:= :=·== =.=.

'=	j

0

."u..'

(D FUEL QUANTITY indicator
The number shows the quantity of fuel in each tank.
0 HI LVL It
This blue light comes on when the system detects a high fuel level. The corresponding refuel valve closes automatically.
Q) REFUEL  VALVES  sel (guarded in NORM)
NORM	Automatic refueling logic controls the refuel valves.
OPEN : Valves open when the MODE SELECT switch is set to REFUEL or DEFUEL position. When the MODE SELECT switch is in the REFUEL position, each refuel/defuel valve closes when the system detects a high level in the associated tank.
SHUT	Valves close.

REFUEL		Refuel valves operate in automatic or in manual mode depending on the position of REFUEL/DEFUEL VALVES switches.
DEFUEL	Refuel/Defuel transfer valve opens.
Refuel valve opens if the associated REFUEL VALVE selector is at OPEN.
® OPEN It
This amber light comes on when the transfer valve is open.
0 TEST SW
When this switch is pressed, the HI LVL lights come on if high level sensors and associated circuits are serviceable.

Note: If tanks are full (HI LVL lights on) during this test, the HI LVL lights go out if
--	high level sensors and associated circuits are serviceable.

LTS : Lights on panel and all 8's on FQI and preselector come on.
0) PRESELECTED  display
This display shows the preselected total fuel quantity in kg (lb) x 1000 (multiply by 1000 to get actual amount).
(!) Preselector rocker switch
Pressing the left or right side of the switch decreases or increases the preselected quantity.
(!) ACTUAL display
This display shows the total fuel on board.
@ END It
- This green light comes on steady when automatic refueling is completed.
- It flashes green if refueling is aborted.

@ CKPT It
Not used.


FUEL QTY
KG X 1000	0
-L,EF-T,,-	-C,TR,,-,	-RIG,HT-,,-,]
2	:,.,: :,	,:. ,,_,

HI VL------•--==,---======HI=.====LV:L=.:-:_=:-_=:- ====	1
'\\ ACTA	o::-i	O LEFT	O CTR	O RIGHT2	I
#	l::i7	REFUEL VALVES---
NORM	NORM	NORM	3
FUEL VALVES	GOPEN GOPEN  GOPEN+-©
OPEN	SHUT	SHUT	SHUT
SHUT	,-	MODE SELECT ----. TEST	1{s\
' 111" r
! 	@=--;--- DEFUEL XFR  _/ OPEN' LTS	NORM	I  (c\ 
?	RESELECTED	REFUEL	ACTUAL	I I	0
13	:, ,-, ,-,	KG X 1000 I	,:,,: I	r,-.	(;;'\
 o:J,.,_ E]..,.,_J, :j?	I	0@
2


(D FUEL QUANTITY indicator
The number shows the quantity of fuel in each tank.
Q) HI LVL light
This blue light comes on, when the system detects a high fuel level. The corresponding refuel valve closes automatically.
Q) REFUEL  VALVES  selector (guarded in NORM)
NORM	Automatic refueling logic controls the refuel valves.
OPEN : Valves open when the MODE SELECT switch is set to the REFUEL or
R	 DEFUEL XFR position. Each refuel valve closes when the system detects a R	high level in the associated tank.
SHUT	Valves close.

REFUEL : Refuel valves operate in automatic or in manual mode depending on the position of REFUEL/DEFUEL VALVES switches.
DEFUEL : Refuel/Defuel transfer valve opens.
Refuel valve opens if the associated REFUEL VALVE selector is at OPEN.
® OPEN It
This amber light comes on when the transfer valve is open.
0 TEST SW
When this switch is pressed, the HI LVL lights come on if high level sensors and associated circuits are serviceable.

Note: If tanks are full (HI LVL lights on) during this test, the HI LVL lights go out if
--	high level sensors and associated circuits are serviceable.

LTS : Lights on panel and all 8's on FQI and preselector come on.
0) PRESELECTED  display
This display shows the preselected total fuel quantity in kg (lb) x 1000 (multiply by 1000 to get actual amount).
(!) Preselector rocker switch
Pressing the left or right side of the switch decreases or increases the preselected quantity.
(!) ACTUAL display
This display shows the total fuel on board.
@ END It
- This green light comes on steady when automatic refueling is completed.
- It flashes green if refueling is aborted.

@ CKPT It
Not used.



--,,----HI.LTV_L	---,-----H-I.--L-VL-.=====	1

'\\ AC1TA	o;-., fA.,A2CT	O LEFT	O CTR	O RIGHT-+---201
#	REFUEL VALVES---
,,- REFUEL VALVES -----	GOPEN  GOPEN  GOPEN+-©
NORM	NORM	NORM
OPEN G	OPEN	SHUT	SHUT	SHUT
NORM	NORM	I

<	FQI  SELECT r7JK..cTR

0	8ON ..		

CTR/ACT '(yA \1	OFF	0

@<!i


1	@	;----:::-l. EFUEL XFR_/ OPEN,LTS	NORM	I 0
:	13	  E;E,: c.	o	K E U oo I	AC   I	(;;'I
::;	 11111	.	Ill..
	E]..,.,_J, :j?	I	0@
2


(D FUEL QUANTITY indicator
The number shows the quantity of fuel in each tank.
Q) HI LVL light
This blue light comes on, when the system detects a high fuel level. The corresponding refuel valve closes automatically.
Q) REFUEL  VALVES  selector (guarded in NORM)
NORM	Automatic refueling logic controls the refuel valves.
OPEN : Valves open when the MODE SELECT switch is set to the REFUEL or DEFUEL XFR position. Each refuel valve closes, when the system detects a high level in the associated tank.
SHUT	Valves close.


(D MODE SELECT  sw (guarded at OFF)
OFF	Refuel system is de-energized. Refuel valves are closed.
REFUEL : Refuel valves operate in automatic or in manual mode depending on the position of REFUEL/DEFUEL VALVES switches.
DEFUEL : Refuel/Defuel transfer valve opens.
Refuel valve opens if the associated REFUEL VALVE selector is at OPEN.
® OPEN It
This amber light comes on when the transfer valve is open.
0 TEST SW
When this switch is pressed, the HI LVL lights come on if high level sensors and associated circuits are serviceable.

Note: If tanks are full (HI LVL lights on) during this test, the HI LVL lights go out if
--	high level sensors and associated circuits are serviceable.

LTS : Lights on panel and all 8's on FQI and preselector come on.
0) PRESELECTED  display
This display shows the preselected total fuel quantity in kg (lb) x 1000 (multiply by 1000 to get actual amount).
(!) Preselector rocker switch
Pressing the left or right side of the switch decreases or increases the preselected quantity.
(!) ACTUAL display
This display shows the total fuel on board.
@ END It
- This green light comes on steady when automatic refueling is completed.
- It flashes green if refueling is aborted.

@ CKPT It
Not used.


@ BATT POWER  toggle sw <,J
ON		When flight crew switches this switch ON momentarily and releases it, HOT BUS 1 supplies the FOi. After completion of the FOi tests (about 40 secondes), the fuel quantity indications appear and the refuel operation can be selected.
The electrical supply is cut off automatically :
- after 10 minutes, if no refuel operation is selected
- or, at the end of refueling
NORM	The FOi is not supplied by batteries




















INTENTIONALLY LEFT BLANK

second), the fuel quantity indications appear and refuel operation can be selected.
The electrical supply is automatically cut off :
- After 10 minutes, if no refuel operation is selected, or
- At the end of refueling.
NORM : The FOi is not supplied by batteries.

@ FOi SELECT  CTR/ACT switch

This selects the tank(s) quantity to be displayed on the CTR tank indicator. CTR + ACT : Center tank and ACT.
CTR	: Center tank only.
ACT	: ACT only.

R	Note : When no ACT is installed, the selector is not active and the CTR tank quantity
R	is always indicated, irrespective of the selector position.




















INTENTIONALLY LEFT BLANK

seconds), the fuel quantity indications appear and refuel operation can be selected.
The electrical supply is automatically cut off :
- After 10 minutes, if no refuel operation is selected, or
- At the end of refueling.
NORM : The FOi is not supplied by batteries.

@ FOi SELECT  CTR/ACT switch

This selects the tank(s) quantity to be displayed on the CTR tank indicator. CTR + ACT : Center tank and ACT 1 and 2.
CTR	Center tank only.
ACT 1	: ACT 1 only.
ACT 2	: ACT 2 only.

R	Note : When no ACT is installed, the selector is not active and the CTR tank quantity
R	is always indicated, irrespective of the selector position.




















INTENTIONALLY  LEFT BLANK





















INTENTIONALLY  LEFT BLANK


























<
<
0
0
<
I
0"'
0

0f--.;
IJ'


I
0
N
"N '
I
0
'
<.>
z

(D Wing pumps indications
In line	 - Green : pump pressure is normal (pump contactor on). "LO"	- Amber : pump pressure is low (pump contactor on). Cross line - Amber : pump contactor is off.
0 CTR tank pumps indications


In line "LO"
Cross line Cross line
- 
Green : pump pressure is normal (pump contactor on).
- Amber : pump pressure is low (pump contactor on).
- Green : pump contactor is off and auto shut off is required.
- Amber : pump contactor is off and auto shut off is not required.





















INTENTIONALLY  LEFT BLANK





0
0


0 






z

0 Wing pumps indications
In line	 - Green : pump pressure is normal (pump contactor on). "LO"	- Amber : pump pressure is low (pump contactor on). Cross line - Amber : pump contactor is off.
0 CTR  tanks valves indications


In line In line
Cross line Cross line


- Green : transfer valve is open.
- Amber : transfer valve position disagrees with switch position (open).
- Green : transfer valve is closed.
- Amber : transfer valve position disagrees with switch position (closed).





















INTENTIONALLY  LEFT BLANK





	FUEL
@)	I

1  --	F.U1+S2ED --
200
KG

2		
CJ)	I	I

©	I	I
0	 I	I

<-  CD...-  -		I  I	0 
--------,--;--------( 2

5030

-21'C	-1


<
<
0
N N
<
0 I	9 >	1-I FOB





14260 KG

N
"N '
I
0
I
"u'
2


0 Wing pump indications
lnline	 - Green : Pump pressure is normal (pump contactor on). "LO"	- Amber : Pump pressure is low (pump contactor on). Crossline	 - Amber : Pump contactor is off.
0 CTR tank pump indications


lnline "LO"
Crossline Crossline


- Green : Pump pressure is normal (pump contactor on).
- Amber : Pump pressure is low (pump contactor on).
- Green : Pump contactor is off, and auto shut-off is required.
- Amber : Pump contactor is off, and auto shut-off is not required.





















INTENTIONALLY  LEFT BLANK

I ECAM FUEL PAGE I
R


F .USED 1
0 I	I	...	100

FUEL KG	F .USED 2
II 1  16360 1 II  ,.  -r-,--'----,---------,	;0

3  1----..---.-- ---	




0,........;---,---L-E-F1..T.

,.

CTR





RIGHT

-----10



"<'
"'

liJliJ..-------,	0

	




z

0 Wing pumps indications
In line	 - Green : pump pressure is normal (pump contactor on). "LO"	- Amber : pump pressure is low (pump contactor on). Cross line - Amber : pump contactor is off.
0 CTR  tanks valves indications


In line In line
Cross line Cross line


- Green : transfer valve is open.
- Amber : transfer valve position disagrees with switch position (open).
- Green : transfer valve is closed.
- Amber : transfer valve position disagrees with switch position (closed).

A®318/A319/A320/A321
TURKISH AIRLINES

FLIGHT CREW OPERATING MANUAL
FUEL
CONTROLS AND INDICATORS
1.28.20	p 7


SEQ 001 I REV 23




















INTENTIONALLY  LEFT BLANK
























THY MSN 2609-2739 2928-2984 3010 3142-3185 3208-3257 3303-3341 3558-3603 3654 3718-3719
3896-3949





@)	I  I 

FUEL

1  --	F.U1+S2ED --
200
KG

2		
CJ)	I	I

©	I   I 
0	 I	I

<-  CD...-  -		I  I	0 
--------,--;--------( 2



I

:<:
"<'
I
0"'
0
I

5030

-21'C	-1

0
N
"N '
I
0
I
"u'
2


0 Wing pump indications
lnline	 - Green : Pump pressure is normal (pump contactor on). "LO"	- Amber : Pump pressure is low (pump contactor on). Crossline	 - Amber : Pump contactor is off.
0 CTR tank pump indications


lnline "LO"
Crossline Crossline


- Green : Pump pressure is normal (pump contactor on).
- Amber : Pump pressure is low (pump contactor on).
- Green : Pump contactor is off, and auto shut-off is required.
- Amber : Pump contactor is off, and auto shut-off is not required.











THY MSN 2609-2739 2928-2984 3010 3142-3185 3208-3257 3303-3341 3558-3603 3654 3718-3719
3896-3949

A®318/A319/A320/A321
TURKISH AIRLINES

FLIGHT CREW OPERATING MANUAL
FUEL
CONTROLS AND INDICATORS
1.28.20	p 7


SEQ 001 I REV 23


















.N,
N I


0 Wing pumps indications
lnline	 - Green : Pump pressure is normal (pump contactor is on). "LO"	- Amber : Pump pressure is low (pump contactor is on). Crossline	 - Amber : Pump contactor is off.
0 CTR  tanks valves indications


lnline lnline

Crossline Crossline


- Green : Transfer valve is open.
- Amber : Transfer valve is open, although commanded closed in automatic or manual mode.
- Green : Transfer valve is closed.
- Amber : Transfer valve is closed, although commanded open in automatic or manual mode.

A®318/A319/A320/A321
TURKISH AIRLINES

FLIGHT CREW OPERATING MANUAL
FUEL
CONTROLS AND INDICATORS
1.28.20	p 7


SEQ 001 l REV 23




















INTENTIONALLY LEFT BLANK
















<
• 43 KG/MIN

i	0  --+-1'"!

F. FLOW
1+2
-FO_B_.	

1_63_60_K_G ,


I
0
.N,
N I








0 Wing pumps indications
lnline	 - Green : Pump pressure is normal (pump contactor on). "LO"	- Amber : Pump pressure is low (pump contactor on). Crossline	 - Amber : Pump contactor is off.
0 CTR  tanks-valve indications


lnline lnline


- Green : Transfer valve is open.
- Amber : Transfer valve is open, whereas commanded closed in automatic or manual mode.

Crossline	- Green : Transfer valve is closed.
Crossline	- Amber : Transfer valve is closed, whereas commanded open in automatic or manual mode.

A®318/A319/A320/A321
TURKISH AIRLINES

FLIGHT CREW OPERATING MANUAL
FUEL
CONTROLS AND INDICATORS
1.28.20	p 7


SEQ 001 I REV 23




















INTENTIONALLY LEFT BLANK
















<




I
0
.N,
N I








0 Wing pumps indications
lnline	 - Green : Pump pressure is normal (pump contactor on). "LO"	- Amber : Pump pressure is low (pump contactor on). Crossline	 - Amber : Pump contactor is off.
0 CTR  tanks valves indications


lnline lnline

Crossline Crossline


- Green : Transfer valve is open.
- Amber : Transfer valve is open, whereas commanded closed in automatic or manual mode.
- Green : Transfer valve is closed.
- Amber : Transfer valve is closed, whereas commanded Open in automatic or manual mode.

Transit	- Amber : Valve is in transit.
0 X feed indications


In line In line
Cross line Cross line Transit


- Green : valve is open.
- Amber : valve is open with pushbutton switch off.
- Green : valve is closed.
- Amber : valve is closed with pushbutton switch ON.
- Amber : valve is in transit.

® Transfer valves indications




5030	I

















<
<
0
0
<
I
0"'
0

One transfer valve is open:
Green LH or RH inner cell tank is at Low Level.
Amber : LH and RH inner cells tanks are not at Low level (associated with ECAM caution).

?oo	 \so3o	I
One transfer valve is in transit.

I
0

N
"N '
I
0
I

'-'
z

Both transfer valves are closed
Green  LH or RH inner cells tanks are not at Low Level.
Amber : LH and RH inner cell tank is at low level (associated with ECAM caution).

0 Fuel temperature indication
This appears when its associated temperature sensor is wet. It is normally green.
Advisory in phases 2 and 6 only, when fuel temperature is :
• above 45° C for inner cell or 55° C for outer cell.
· below - 40° C.
It becomes amber, and ECAM displays a caution, if the temperature goes above the high limit or below the low limit.

(D Fuel quantity indication





- Normally green
- An amber line appears across the last two digits when FQI is inaccurate (Refer to 3.04.28). The outer indication is boxed amber if both transfer valves fail to open when inner at low level.
R   - The center tank indication is boxed amber if both center tank pumps are failed or R	switched OFF.
- Advisory in flight phases 2 and 6, when difference between fuel quantities in the two wings is greater than 1500 kg (3300 lb). The indications of the wing inner and outer tanks with the highest fuel level pulses.
R © Fuel on board (FOB) indication It is normally green.
An amber line appears across the last two digits when the FQI is inaccurate (Refer to
3.04.28).
The indication is boxed amber if :
- Center tank pumps failed or switched OFF
- Both transfer valves fail to open when inner cell at low level.
0 Fuel used indication F.USED 1
3100
- The engine identification number is amber when the engine is below idle, and it is white when it is at or above idle.
- The fuel used indication is green from flight phase 2 until electrical power is cut off at the end of the flight. It is automatically reset when the engine is started on ground.

0 X feed indications


In line In line
Cross line Cross line Transit


- Green : valve is open.
- Amber : valve is open with pushbutton switch off.
- Green : valve is closed.
- Amber : valve is closed with pushbutton switch ON.
- Amber : valve is in transit.


This appears when its associated temperature sensor is wet.
- It is normally green.
- It is advisory in phases 2 and 6 only, if the fuel temperature is above 45° C or below
- 40° C.
- It becomes amber, and ECAM displays a caution, if the temperature goes above the high limit or below the low limit.
® Fuel quantity indication

[	5780	i
It is normally green.
- The wing tank indication turns amber if a wing tank overflows.
- An amber line appears accross the last two digits when FQI is inaccurate (See
3.04.28).
- The center tank indication is boxed amber if both transfer valves fail in the closed position.
- It is advisory in phases 2 and 6, when difference between fuel quantities in the two wings is greater than 1500 kg (3300 lbs). The indication for the wing with the higher fuel level pulses.
R 0) Fuel on board (FOB) indication It is normally green.
An amber line appears across the last two digits when the FQI is inaccurate (Refer to
3.04.28).
The indication is boxed amber when both center tank transfer valves fail in closed position.
© Fuel used indication F.USED 1
3100
- The engine identification number is amber when the engine is below idle, and it is white when it is at or above idle.
- The fuel used indication is green from flight phase 2 until electrical power is cut off at the end of the flight. It is automatically reset when the engine is started on ground.

0 X feed indications


In line In line
Cross line Cross line Transit


- Green : valve is open.
- Amber : valve is open with pushbutton switch off.
- Green : valve is closed.
- Amber : valve is closed with pushbutton switch ON.
- Amber : valve is in transit.


This appears when its associated temperature sensor is wet.
- It is normally green.
- It is advisory in phases 2 and 6 only, if the fuel temperature is above 45° C or below
- 40° C.
- It becomes amber, and ECAM displays a caution, if the temperature goes above the high limit or below the low limit.
® Fuel quantity indication

[	5780	I
It is normally green.
- The wing tank indication turns amber if a wing tank overflows.
- An amber line appears across the last two digits when FQI is inaccurate (See
3.04.28).
- The center tank indication is boxed amber if both transfer valves fail in the closed position.
- It is advisory in phases 2 and 6, when difference between fuel quantities in the two wings is greater than 1500 kg (3300 lb). The indication for the wing with the higher fuel level pulses.
0) Fuel on board (FOB) indication It is normally green.
An amber line appears across the last two digits when the FQI is inaccurate (Refer to
3.04.28).
It is boxed amber when both center tank transfer valves fail in closed position.
(!) Fuel used indication
- The engine identification number is amber when the engine is below idle, and it is white when it is at or above idle.
- The fuel used indication is green from flight phase 2 until electrical power is cut off at the end of the flight. It is automatically reset when the engine is started on ground.
0) ACT to center tank fuel transfer indication
It is normally green. A triangle indicates that the ACT fuel transfer is initiated into the center tank.



lnline lnline Crossline Transit
- 
Green : The valve is open.
- Amber : The valve is open, with the ENG MASTER switch OFF.
- Amber : The ENG valve is fully closed.
- Amber : The valve is in transit.

0 X feed indications


lnline lnline Crossline Crossline Transit


- Green : The valve is open.
- Amber : The valve is open, with the X Feed pushbutton off.
- Green : The valve is closed.
- Amber : The valve is closed with X feed pushbutton ON.
- Amber : The valve is in transit.

® Transfer valve indications Outer to Inner Transfer

I>	The triangle is green, during a transfer to inner.
? The triangle is in solid amber, when the valves are open, while commanded closed.
I>	The triangle is in amber, when a valve is in transit.

® APU indications
APU <J--: "APU" is in white and the triangle is in green : APU valve is open. APU +-:	"APU" and the triangle are in amber : APU valve is open,
with APU Fire pushbutton out, or APU MASTER switch OFF.

APU APU <l

"APU" is in amber : APU valve is closed and the APU Fire pushbutton is out, or APU MASTER Switch ON.
"APU" and the triangle are in white, the APU valves is closed.













THY MSN 2609-2739 2928-2984 3010 3142-3185 3208-3257 3303-3341 3558-3603 3654 3718-3719
3896-4629


This appears, when its associated temperature sensor is wet. It is normally green. Advisory only in phases 2 and 6, when fuel temperature is :
• Above 45° C for the inner cell, or 55° C for the outer cell.
· Below - 40° C.
It becomes amber, and the ECAM displays a caution, if the temperature goes above the high limit or below the low limit.
© Fuel quantity indication
§[:] 
- It is normally in green.
- An amber line appears across the last two digits, when the FQI is inaccurate (Refer to 3.04.28). The outer indication is boxed amber, if both transfer valves fail to open when the inner is at low level.
- The center tank indication is boxed amber, if both center tank pumps are failed, or are switched OFF.
- Advisory in flight phases 2 and 6, when the difference between the fuel quantities in the two wings is greater than 1500 kg (3300 lb). The wing inner and outer tank indications pulse with the highest fuel level.
0 Fuel on Board (FOB) indication It is normally in green.
An amber line appears across the last two digits when the FQI is inaccurate (Refer to
3.04.28).
The indication is half-boxed in amber, if :
- Center tank pumps fail, or are switched OFF.
- Both transfer valves fail to open, when the inner cell is at low level.
@ Fuel Used indication F.USED 1
3100
- The engine identification number is in amber, when the engine is below idle. It is in white, when it is at, or above, idle.
- The fuel used indication is in green from flight phase 2, until electrical power is cut
off at the end of the flight. It is automatically reset, when the engine is started on ground.
THY MSN 2609-2739  2928-2984 3010 3142-3185 3208-3257 3303-3341 3558-3603 3654 3718-3719
3896-4629



lnline lnline Cross line Transit
- 
Green : The valve is open.
- Amber : The valve is open with ENG MASTER switch OFF.
- Amber : The ENG valve is fully closed.
- Amber : The valve is in transit.

0 X-feed indications


In line In line
Cross line Cross line Transit


- Green : The valve is open.
- Amber : The valve is open with the X-feed pushbutton off.
- Green : The valve is closed.
- Amber : The valve is closed with the X-feed pushbutton ON.
- Amber : The valve is in transit.

® TRANSFER  Valve indications : Center to inner transfer

Displayed in Green, when there is a center to inner tank transfer•
._J Displayed in amber, when the tranfer valve is open, whereas commanded closed in automatic
or manual mode.

® APU Indications
APU <J-: APU is in white and the triangle is in green: the APU valve is open. APU ._: APU and solid triangle are in amber : the APU valve is open,
with the APU Fire Pushbutton out, or the APU MASTER switch OFF.

APU APU <l

APU is in amber : APU valve is closed and APU Fire pushbutton is out/ or APU MASTER switch ON.
APU and triangle are in white: the APU valve is closed.


- It is normally in green.
- It is advisory in phases 2 and 6 only, if the fuel temperature is above 45° C, or below
- 40° C.
- It becomes amber, and the ECAM displays an associated caution, if the temperature goes above the high limit, or below the low limit.
® Fuel quantity indication

c- I \ ,,::J
It is normally in green.
- The wing tank indication turns amber, if a wing tank overflows.
- An amber line appears across the last two digits, when the FQI is inaccurate (See
3.04.28).
- The center tank indication is boxed amber, if both transfer valves fail in the closed position.
- It is advisory in phases 2 and 6, when the difference between the fuel quantities in the two wings is greater than 1500 kg (3300 lb). The indication for the wing with the higher fuel level pulses.
0 Fuel On Board (FOB) indication It is normally in green.
An amber line appears across the last two digits, when the FQI is inaccurate (Refer to
3.04.28).
It is half boxed amber, when both center tank transfer valves fail in the closed position.

@ Fuel Used indication
- The engine identification number is in amber, when the engine is below idle. It is in white when it is at, or above, idle.
- The fuel used indication is green from flight phase 2, until electrical power is cut off at the end of the flight. It is automatically reset, when the engine is started on ground.



lnline lnline Cross line Transit
- 
Green : The valve is open.
- Amber : The valve is open with ENG MASTER switch OFF.
- Amber : The ENG valve is fully closed.
- Amber : The valve is in transit.

0 X-feed indications


In line In line
Cross line Cross line Transit


- Green : The valve is open.
- Amber : The valve is open with the X-feed pushbutton off.
- Green : The valve is closed.
- Amber : The valve is closed with the X-feed pushbutton ON.
- Amber : The valve is in transit.

® TRANSFER  Valve indications : Center to inner transfer

Displayed in Green, when there is a center to inner tank transfer•
._J Displayed in amber, when the tranfer valve is open, whereas commanded closed in automatic
or manual mode.

® APU Indications
APU <J-: APU is in white and the triangle is in green: the APU valve is open. APU ._: APU and solid triangle are in amber : the APU valve is open,
with the APU Fire Pushbutton out, or the APU MASTER switch OFF.

APU APU <l

APU is in amber : APU valve is closed and APU Fire pushbutton is out/ or APU MASTER switch ON.
APU and triangle are in white: the APU valve is closed.


- It is normally in green.
- It is advisory in phases 2 and 6 only, if the fuel temperature is above 45° C, or below
- 40° C.
- It becomes amber, and the ECAM displays an associated caution, if the temperature goes above the high limit, or below the low limit.
® Fuel quantity indication


1,0.2.. j
? 4	ACT QUANTITIES <fl
It is normally in green.
- The wing tank indication turns amber, if a wing tank overflows.
- An amber line appears across the last two digits, when the FQI is inaccurate (See
3.04.28).
- The center tank indication is boxed amber, if both transfer valves fail in the closed position.
- It is advisory in phases 2 and 6, when the difference between the fuel quantities in the two wings is greater than 1500 kg (3300 lb). The indication for the wing with the higher fuel level pulses.
0 Fuel On Board (FOB) indication It is normally in green.
An amber line appears across the last two digits, when the FQI is inaccurate (Refer to
3.04.28).
It is half boxed amber, when both center tank transfer valves fail in the closed position.

@ Fuel Used indication
- The engine identification number is in amber, when the engine is below idle. It is in white when it is at, or above, idle.
- The fuel used indication is green from flight phase 2, until electrical power is cut off at the end of the flight. It is automatically reset, when the engine is started on ground.

A®318/A319/A320/A321
TURKISH AIRLINES

FLIGHT CREW OPERATING MANUAL
FUEL
CONTROLS AND INDICATORS
1.28.20	P 10a


SEQ 303 I REV 36

@ Fuel Flow indication
The Total Fuel Flow is displayed in kg/mn.
- It is normally in green.
- It is replaced by an amber "XX", if there is no valid data.

A®318/A319/A320/A321
TURKISH AIRLINES

FLIGHT CREW OPERATING MANUAL
FUEL
CONTROLS AND INDICATORS
1.28.20	P 10a


SEQ 400 I REV 38

@ ACT to center tank fuel transfer indication
It is normally in green. ACT quantity is displayed in a grey box; This box becomes amber in case of a transfer fault.
A triangle indicates that the ACT fuel transfer to the center tank has started :
I:::.	: Green, when the automatic transfer begins.
.&.	: Fully green, when the manual transfer begins.
@ Fuel Flow indication
- It is normally in green.
- It is replaced by amber XX if there is no valid data.

































THY MSN 3637 3673-3688 3738



0 Total fuel indication
An amber half box appears around FOB when the quantity shown is not all usable
R	(intercell transfer valve failure or loss of center tank pumps).
In case of degraded data, the last two significant digits have dashes across them (refer to 3.04.28).
0 Memo indications : (green)

0"'	Q	z
"'	+'	Q
"I '	!	LI.	+'


0	Cl)
I

LI.	.8..

:c
Cl)	IX

0	!	!!	!!	LI.

!!	LI.

N	0
N	'-'	:..::	I-	'-'	:..::

I	w	.I-..	.I.-.

.L..I..

;:;	...J	Cl)	Cl)
.:,

5il

...J

5il	N

.u..

1	2	3	4	5	6	7	8	9	10



E / WO : FAILURE TITLE
conditions

 AURAL WARNING

MASTER LIGHT
SD PAGE CALLED
  LOCAL WARNING
FLT PHASE INHIB
L(R) TK PUMP 1 + 2 LO PR












SINGLE CHIME












MASTER CAUT












FUEL
L(R) TK PUMP 1 + 2
FAULT It



3, 4, 5,
7, 8 *
CTR TK PUMP 1(2) LO PR



  CTR TK PUMP 1(2)
FAULT It

CTR TK PUMPS LO PR



CTR TK PUMP
FAULT Its

CTR TK PUMPS OFF
CTR TK pb at OFF with no FAULT



OFF It on CTR TK
PUMP ob
1, 3, 4,
5, 7, 8,
9, 10
AUTO FEED FAULT
(CTR TK > 250 kg (550 lb) and
Lor R WING TK < 5000 kg (11000 lb)
-DR




MODE SEL FAULT Its
--

3, 4,
5, 8
(CTR TK pumps do not stop after slat extension or CTR
TK low level)



NIL

L(R) WING TK LO LVL (750 kg - 1650 lb)








NIL

3, 4, 5,
7, 8, 9
L + R WING TK LO LVL
Low level detected in both wing inner cells (remaining flight time is about 30 minutes)





L(R) OUTER TK HI TEMP or L(R) INNER TK HI TEMP
Fuel temp above :
in outer cell above 60° in inner cell above 54°






3, 4, 5,
7, 8
L(R) XFR VALVE CLOSED
both transfer valves fail to open after inner cell low level





ENG 1(2) LP VALVE OPEN
valve disagree in open position




4, 5,
7, 8
* inhibited if pump selected OFF





	





0


0

0


0


0Total fuel indication
An amber half box appears around FOB when the quantity shown is not all usable (both center tank transfer valves fail in closed position).
In case of degraded data, the last two significant digits have dashes across them (refer to 3.04.28).
0 Memo indications : (green)

""''	Q	z

"Cl'
N

+'	Q

"I '	!	LI.	+'


Cl	Cl)
I

LI.	.8..

:c
Cl)	IX

Cl	!

!!	!!	LI.

!!	LI.

N	0
N	'-'	:..::	I-	'-'	:..::

I	w	.I-..	.I.-.

.L..I..

;:;	...J	Cl)	Cl)
.:,

5il

...J

5il	N

.u..
R

1	2	3	4	5	6	7	8	9	10


























* inhibited if pump selected OFF





	





0


0

0


0


0Total fuel indication
An amber half box appears around FOB when the quantity shown is not all usable (both center tank transfer valves fail in closed position).
In case of degraded data, the last two significant digits have dashes across them (refer to 3.04.28).
0 Memo indications : (green)

0"'	Q	z
"N '	+'	Q
"I '	!	LI.	+'


0	Cl)
I

LI.	.8..

:c
Cl)	IX

0	!	!!	!!	LI.

!!	LI.

N	0
N	'-'	:..::	I-	'-'	:..::

I	w	.I-..	.I.-.

.L..I..

;:;	...J	Cl)	Cl)
.:,

5il

...J

5il	N

.u..
R

1	2	3	4	5	6	7	8	9	10


























* inhibited if pump selected OFF




			



<
<
0
N N
<
I

0
I
0
.N,
N I


0
"u''
z

0


0




Total fuel indication

A half amber box appears around FOB, when the displayed quantity is not all usable (intercell transfer valve failure, or loss of center tank pumps).
In case of degraded data, the last two significant digits are dashed (refer to 3.04.28). Memo indications : (green)

"'
"'
Cl	+'	Q
"I '	!	LI.	+'

Cl)

8	:c

Cl	LI.	...

Cl)	IX

Cl	!!	!!	0	!!	LI.
N	'-'	:..::	I-	'-'	:..::

I	w	.I-..	.I.-.

.L..I..

;:;	...J	Cl)	Cl)
.:,

5il

...J

5il	N

.u..

1	2	3	4	5	6	7	8	9	10



E / WO : FAILURE TITLE
conditions

 AURAL WARNING

MASTER LIGHT
SD PAGE CALLED
  LOCAL WARNING
FLT PHASE INHIB
L(R) TK PUMP 1 + 2 LO PR













SINGLE













MASTER

L(R) TK PUMP 1 + 2
FAULT It



3, 4, 5,
7, 8 *
CTR TK PUMP 1(2) LO PR



  CTR TK PUMP 1(2)
FAULT It

CTR TK PUMPS LO PR



CTR TK PUMP
FAULT Its

CTR TK PUMPS OFF
CTR TK pb at OFF with no FAULT



OFF It on CTR TK
PUMP ob
1, 3, 4,
5, 7, 8,
9, 10
AUTO FEED FAULT
(CTR TK > 250 kg (550 lb) and
Lor R WING TK < 5000 kg (11000 lb)
-DR




MODE SEL FAULT Its
--

3, 4,
5, 8
(CTR TK pumps do not stop after slat extension or CTR TK low level)
CHIME
CAUT
FUEL
NIL

L(R) WING TK LO LVL (750 kg - 1650 lb)









NIL
3, 4, 5,
7, 8, 9
L + R WING TK LO LVL
Low level detected in both wing inner cells (remaining flight time is about 30 min)





L(R) OUTER TK HI TEMP or L(R) INNER TK HI TEMP
Fuel temp above :
in outer cell above 55° on ground in outer cell above 60° in flight
in inner cell above 45° on ground in inner cell above 54° in flight







3, 4, 5,
7, 8
L(R) XFR VALVE CLOSED
both transfer valves fail to open after inner cell low level





ENG 1(2) LP VALVE OPEN
valve disagree in open position




4, 5,
7, 8
* inhibited if pump selected OFF




			



<
<
0
N N
<
I

0
I
0
.N,
N I


0
"u''
z

0


0




Total fuel indication

A half amber box appears around FOB, when the displayed quantity is not all usable (intercell transfer valve failure, or loss of center tank pumps).
In case of degraded data, the last two significant digits are dashed (refer to 3.04.28). Memo indications : (green)

0"'	Q	z
"'	+'	Q
"I '	!	LI.	+'


0	Cl)
I

LI.	.8..

:c
Cl)	IX

0	!	!!	!!	LI.

!!	LI.

N	0
N	'-'	:..::	I-	'-'	:..::

I	w	.I-..	.I.-.

.L..I..

;:;	...J	Cl)	Cl)
.:,

5il

...J

5il	N

.u..

1	2	3	4	5	6	7	8	9	10



E / WO : FAILURE TITLE
conditions

 AURAL WARNING

MASTER LIGHT
SD PAGE CALLED
  LOCAL WARNING
FLT PHASE INHIB
L(R) TK PUMP 1 + 2 LO PR












SINGLE CHIME












MASTER CAUT












FUEL
L(R) TK PUMP 1 + 2
FAULT It



3, 4, 5,
7, 8 *
CTR TK PUMP 1(2) LO PR



  CTR TK PUMP 1(2)
FAULT It

CTR TK PUMPS LO PR



CTR TK PUMP
FAULT Its

CTR TK PUMPS OFF
CTR TK pb at OFF with no FAULT



OFF It on CTR TK
PUMP ob
1, 3, 4,
5, 7, 8,
9, 10
AUTO FEED FAULT
(CTR TK > 250 kg (550 lb) and
Lor R WING TK < 5000 kg (11000 lb)
-DR




MODE SEL FAULT Its
--

3, 4,
5, 8
(CTR TK pumps do not stop after slat extension or CTR
TK low level)



NIL

L(R) WING TK LO LVL (750 kg - 1650 lb)








NIL

3, 4, 5,
7, 8, 9
L + R WING TK LO LVL
Low level detected in both wing inner cells (remaining flight time is about 30 minutes)





L(R) OUTER TK HI TEMP or L(R) INNER TK HI TEMP
Fuel temp above :
in outer cell above 60° in inner cell above 54°






3, 4, 5,
7, 8
L(R) XFR VALVE CLOSED
both transfer valves fail to open after inner cell low level





ENG 1(2) LP VALVE OPEN
valve disagree in open position




4, 5,
7, 8
* inhibited if pump selected OFF




			



<
<
0
N N
<
I

0
I
0
.N,
N I


0
"u''
z

0


0




Total fuel indication

A half amber box appears around FOB, when the displayed quantity is not all usable (intercell transfer valve failure, or loss of center tank pumps).
In case of degraded data, the last two significant digits are dashed (refer to 3.04.28). Memo indications : (green)

0	..,	:z
.	UJ	":,",:	...	.......	0

I-	0
'	(/)	I-

...

8
LI'\

:=I:>
:z	(/)

."U.J".

0	"iiE"

:<z.!:l

:<z.!:l

..,

0	..,

:<z.!:l	I-


0	UJ	UJ

.'.r.


:I:

UJ	<

'	J

.(./)..	.(./)..

....	0	:z	::;;:

N	UJ	I-	I-
0	UJ
'

=>	0	:z
....J	I-	N	LI'\

z	2	3	4	5	6	7	8	9	10

E / WO : FAILURE TITLE
conditions
 AURAL WARNING
MASTER LIGHT
SD PAGE
CALLED
  LOCAL WARNING
  FLT PHASE
INHIB

F. USED /FOB DISAGREE

SINGLE CHIME

MASTER CAUT

FUEL

NIL
1, 2, 3,
4, 5,
6 * *,
7, 8, 9,
10
L(R) TK PUMP 1 + 2 LO PR












SINGLE












MASTER












FUEL
L(R) TK PUMP 1 + 2
FAULT It



3, 4, 5,
7, 8 *
CTR TK PUMP 1(2) LO PR



  CTR TK PUMP 1(2)
FAULT It

CTR TK PUMPS LO PR



CTR TK PUMP FAULT Its

CTR TK PUMPS OFF
CTR TK pb at OFF with no FAULT



OFF It on CTR TK
PUMP ob
1, 3, 4,
5, 7, 8,
9, 10
AUTO FEED FAULT
(CTR TK > 250 kg (550 lb) and
Lor R WING TK < 5000 kg (11000 lb)
-DR




MODE SEL FAULT Its
--

3, 4,
5, 8
(CTR TK pumps do not stop after slat extension or CTR TK low level)
CHIME
GAUT

NIL

L(R) WING TK LO LVL (750 kQ - 1650 lb)








NIL
3, 4, 5,
7, 8, 9
L + R WING TK LO LVL
Low level detected in both wing inner cells (remaininQ fliQht time is about 30 minutes)





L(R) OUTER TK HI TEMP or L(R) INNER TK HI TEMP
Fuel temp above :
In outer cell, above 60°. In inner cell, above 54°.






3, 4, 5,
7, 8
L(R) XFR VALVE CLOSED
Both transfer valves fail to open after inner cell low level.





ENG 1(2) LP VALVE OPEN
Valve disagree in open position.




4, 5,
7, 8

* Inhibited, if the pump is selected OFF.
** Inhibited, only if the APU has been running for more than 30 minutes after takeoff.










<
<
"'
"<'
I


----<
-----,0


0
I
0
N
"N '
I

0
I
"u'
2


0 Total fuel indication
An half amber box appears around FOB, when the displayed quantity is not all usable (both center tank transfer valves fail in the closed position).
In case of degraded data, the last two significant digits are dashed (Refer to 3.04.28).
0 Memo indications : (green)

0"'	Q	z
"N '	+'	Q
"I '	!	LI.	+'


0	Cl)
I

LI.	.8..

:c
Cl)	IX

0	!	!!	!!	LI.

!!	LI.

N	0
N	'-'	:..::	I-	'-'	:..::

I	w	.I-..	.I.-.

.L..I..

;:;	...J	Cl)	Cl)
.:,

5il

...J

5il	N

.u..
R

1	2	3	4	5	6	7	8	9	10


























* inhibited if pump selected OFF










<
<
"'
"<'
I


---1
-----,0


0
I
0
N
"N '
I

0
I
"u'
2


0 Total fuel indication
An half amber box appears around FOB, when the displayed quantity is not all usable (both center tank transfer valves fail in the closed position).
In case of degraded data, the last two significant digits are dashed (Refer to 3.04.28).
0 Memo indications : (green)

<
<
0
""''


0
LU	0::	.....

.....	z

<	0::	c..	LL	LL
N

0
I-
::::>

;;	V)	0

:I:
V)


0::

I	0::	LL
0	3	<.!l	<.!l	LL

."...'	<.!l	LU

N	c..	z	z

.....	0	0

.....

z	LL

"N '	u

LU	LU

:,.:

I	:I:
I-

:,.:

LU	<

;;	...J

.V.)..	.V.)..

....	5	z


I	LU
"u'

a3	...J	I-	a3	N	"'

:z	2	3	4	5	6	7	8	9	10
R






















* Inhibited, if the pump is selected OFF.


E / WO : FAILURE TITLE
conditions

 AURAL WARNING

MASTER LIGHT
SD PAGE
CALLED

  LOCAL WARNING
  FLT PHASE
INHIB
APU LP VALVE FAULT
Valve position disaaree





NIL





NIL





FUEL

NIL





3, 4, 5,
7, 8 *
L(R) OUTER TK LO TEMP, or L(R) INNER TK LO TEMP
Fuel temperature < approx. - 43°C





L(R) TK PUMP 1(2) LO PR



  L(R) TK PUMP 1(2)
FAULT It

L(R) XFR VALVE OPEN
Either transfer valve opens before inner tank reaches low level





NIL

X FEED VALVE FAULT
Valve position disaQree





FQI CH 1 (2) FAULT






  *  PUMP LO  PR  is inhibited, if the pump is selected OFF in phases 1 and 10. I MEMO  DISPLAY  I
- OUTR TK FUEL XFRD appears in green, if at least one transfer valve is open in one wing tank.
- CTR TK FEEDG appears in green, if at least one center tank pump is energized.
- FUEL X FEED appears in green, if the fuel X FEED valve pushbutton is ON, and the X FEED valve is not fully closed. It appears in amber in flight phases 3, 4, or 5.
R	- REFUELG appears in green, when the door of the refuel control panel on the fuselage or R		on the wing	is open.




















THY MSN 1856-2739 2928-2984 3010 3142-3185 3208-3257 3303-3341 3558-3603 3654 3718-3719
3896-3949


E / WO : FAILURE TITLE
conditions

 AURAL WARNING

MASTER LIGHT
SD PAGE
CALLED

  LOCAL WARNING
  FLT PHASE
INHIB
APU LP VALVE FAULT
Valve position disaaree



NIL



NIL



FUEL

NIL



3, 4, 5,
7, 8 *
L(R) WING TK LO TEMP
Fuel temperature < - 44°C





L(R) TK PUMP 1(2) LO PR



L(R) TK PUMP 1(2) FAULT It

X FEED VALVE FAULT
Valve oosition disaaree



NIL

FQI CH 1 (2) FAULT






  *  PUMP LO  PR  is inhibited, if the pump is selected OFF in phases 1 and 10. I M  EMO  DISPLAY  I
- FUEL X FEED appears in green, if the fuel X FEED valve pushbutton is ON, and the X FEED valve is not fully closed. It appears in amber in flight phases 3, 4, or 5.
R	- REFUELG appears in green, when the door of the refuel control panel on the fuselage or R		on the wing	is open.
- FOB BELOW 3 T appears in green, when the fuel quantity in either wing is below 1500 kilograms.


E / WO : FAILURE TITLE
conditions

 AURAL WARNING

MASTER LIGHT
SD PAGE
CALLED

  LOCAL WARNING
  FLT PHASE
INHIB
APU LP VALVE FAULT
Valve position disaaree







NIL







NIL





FUEL

NIL





3, 4, 5,
7, 8 *
L(R) OUTER TK LO TEMP, or L(R) INNER TK LO TEMP
Fuel temperature < approx. - 43°C





L(R) TK PUMP 1(2) LO PR



  L(R) TK PUMP 1(2)
FAULT It

L(R) XFR VALVE OPEN
Either transfer valve opens before inner tank reaches low level






NIL

X FEED VALVE FAULT
Valve position disaQree





FQI CH 1 (2) FAULT





IDG 1(2) COOL FAULT
Non-Return Valve 1(2) has failed
?



NIL

2 to 9
LO LVL ET FAULT
Both low level sensors have failed




3, 4, 5
7, 8, 9

  * PUMP LO  PR  is inhibited, if the pump is selected OFF in phases 1 and 10. I MEMO  DISPLAY  I
- OUTR TK FUEL XFRD appears in green, if at least one transfer valve is open in one wing tank.
- CTR TK FEEDG appears in green, if at least one center tank pump is energized.
- FUEL X FEED appears in green, if the fuel X FEED valve pushbutton is ON, and the X FEED valve is not fully closed. It appears in amber in flight phases 3, 4, or 5.
- REFUELG appears in green, when the door of the refuel control panel on the fuselage or on the wing	is open.


E / WO : FAILURE TITLE
conditions

 AURAL WARNING

MASTER LIGHT
SD PAGE
CALLED

  LOCAL WARNING
  FLT PHASE
INHIB
APU LP VALVE FAULT
Valve position disaaree



NIL



NIL







FUEL

NIL



3, 4, 5,
7, 8 *
L(R) WING TK LO TEMP
Fuel temperature < - 44°C





L(R) TK PUMP 1(2) LO PR



L(R) TK PUMP 1(2) FAULT It

X FEED VALVE FAULT
Valve oosition disaaree



NIL

FQI CH 1 (2) FAULT





ACT XFR FAULT
ACT quantity > 250 kg and CTR TK quantity < 3000 kQ


SINGLE CHIME


MASTER CAUT

ACT FAULT
light


3, 4, 5, 7,
8
ACT UNUSABLE
Automatic and manual ACT transfer modes are failed, due to transfer valve failure




NIL

ACT PUMP LO PR
ACT pump low pressure






  * PUMP LO  PR  is inhibited, if the pump is selected OFF in phases 1 and 10. I MEMO  DISPLAY  I
- FUEL X FEED appears in green, if the fuel X FEED valve pushbutton is ON, and the X FEED valve is not fully closed. It appears in amber in flight phases 3, 4, or 5.
R	- REFUELG appears in green, when the door of the refuel control panel on the fuselage or R		on the wing	is open.
- FOB BELOW 3 T appears, when the fuel quantity in either wing is below 1500 kg.


E / WO : FAILURE TITLE
conditions

 AURAL WARNING

MASTER LIGHT
SD PAGE
CALLED

  LOCAL WARNING
  FLT PHASE
INHIB
APU LP VALVE FAULT
Valve position disaaree





NIL





NIL



FUEL

NIL



3, 4, 5,
7, 8 *
L(R) WING TK LO TEMP
Fuel temperature < - 44°C





L(R) TK PUMP 1(2) LO PR



L(R) TK PUMP 1(2) FAULT It

X FEED VALVE FAULT
Valve oosition disaaree





NIL

FQI CH 1 (2) FAULT





IDG 1(2) COOL FAULT
Non-Return Valve 1(2) has failed
?



NIL

2 to 9
LO LVL ET FAULT
Both low level sensors have failed




3, 4, 5
7, 8, 9

  * PUMP LO  PR  is inhibited, if the pump is selected OFF in phases 1 and 10. I MEMO  DISPLAY  I
- FUEL X FEED appears in green, if the fuel X FEED valve pushbutton is ON, and the X FEED valve is not fully closed. It appears in amber in flight phases 3, 4, or 5.
- REFUELG appears in green, when the door of the refuel control panel on the fuselage or on the wing	is open.
- FOB BELOW 3 T appears in green, when the fuel quantity in either wing is below 1500 kilograms.


E / WO : FAILURE TITLE
conditions

 AURAL WARNING

MASTER LIGHT
SD PAGE
CALLED

  LOCAL WARNING
  FLT PHASE
INHIB
APU LP VALVE FAULT
Valve position disaaree



NIL



NIL



FUEL

NIL



3, 4, 5,
7, 8 *
L(R) WING TK LO TEMP
Fuel temperature < - 44°C





L(R) TK PUMP 1(2) LO PR



L(R) TK PUMP 1(2) FAULT It

X FEED VALVE FAULT
Valve oosition disaaree



NIL

FQI CH 1 (2) FAULT





ACT XFR FAULT
ACT quantity > 250 kg and CTR TK quantity < 3000 kQ

SINGLE CHIME

MASTER CAUT

FUEL
ACT FAULT
light

3, 4, 5, 7,
8
ACT PUMP LO PR
ACT pump low pressure



NIL


  * PUMP LO  PR  is inhibited, if the pump is selected OFF in phases 1 and 10. I MEMO  DISPLAY  I
- FUEL X FEED appears in green, if the fuel X FEED valve pushbutton is ON, and the X FEED valve is not fully closed. It appears in amber in flight phases 3, 4, or 5.
     - REFUELG appears in green, when the door of the refuel control panel door on the R	fuselage or on the wing	is open.
- FOB BELOW 3 T appears, when the fuel quantity in either wing is below 1500 kg.

R






















(1) HOT BUS supplies power during refueling on battery.
(2) This occurs if DC BUS 1 is lost.
(3) In smoke configuration (GEN 1 LINE pushbutton OFF), inner tank pumps 1 are supplied directly by IDG (instead of AC 1) and pump relays by DC ESS (instead of DC 1).












THY MSN 1856-2739 2928-2984 3010 3142-3185 3208-3257 3303-3341 3558-3603 3654 3718-3719
3896-4629

R



















(1) HOT BUS supplies power during refueling on battery.
(2) This occurs if DC BUS 1 is lost.
(3) In smoke configuration (GEN 1 LINE pushbutton OFF), wing tank pumps 1 are supplied directly by IDG (instead of AC 1) and pump relays by DC ESS (instead of DC 1).



NORM
EMER ELEC

AC
DC
 AC ESS
 DC ESS
HOT
FQI
CH 1



SHED
(1)

CH 2

DC2


(1)

 WING TANK PUMPS
L 1
ACl (3)
?Cl

X (2) (3)


L 2
AC2
DC2




R 1
ACl (3)
?Cl

X (2) (3)


R 2
AC2
DC2



CROSS FEED VALVE
MOT 1



SHED


MOT 2

DC2




ENGINE LP VALVES
ENG 1 MOT 1



X


ENG 1 MOT 2

DC2




ENG 2 MOT 1



X


ENG 2 MOT 2

DC2



  CTR TK XFR VALVES


?Cl
DC2





APU

PUMP


AC STAT INV
or
AC ESS SHED


HOT 1

LP VALVE

DC BAT


HOTl
REFUEL VALVES

DC2


(1)



ACT
PUMP
AC2





XFR VALVE

?Cl


(1)

AIR SHUT OFF VALVE

?Cl


(1)

ACT 1 INLET VALVE

?Cl


(1)

ACT 1 VENT VALVE

?Cl


(1)

ACT 2 INLET VALVE

?Cl


(1)

ACT 2 VENT VALVE

?Cl


(1)

(1) HOT BUS supplies power during refueling on battery.
(2) This occurs, if DC BUS 1 is lost.
(3) In smoke configuration (GEN 1 LINE pushbutton OFF), wing tank pumps 1 are supplied directly by IDG (instead of AC 1) and pump relays by DC ESS (instead of DC 1).



NORM
EMER ELEC

AC
DC
 AC ESS
 DC ESS
HOT
FQI
CH 1



SHED
(1)

CH 2

DC2


(1)

 WING TANK PUMPS
L 1
ACl (3)
?Cl

X (2) (3)


L 2
AC2
DC2




R 1
ACl (3)
?Cl

X (2) (3)


R 2
AC2
DC2



CROSS FEED VALVE
MOT 1



SHED


MOT 2

DC2




ENGINE LP VALVES
ENG 1 MOT 1



X


ENG 1 MOT 2

DC2




ENG 2 MOT 1



X


ENG 2 MOT 2

DC2



  CTR TK XFR VALVES


?Cl
DC2





APU

PUMP


AC STAT INV
or
AC ESS SHED


HOT 1

LP VALVE

DC BAT


HOTl
REFUEL VALVES

DC2


(1)


ACT
PUMP
AC2





INLET VALVE




(1)

XFR VALVE

DC2


(1)

AIR SHUT OFF VALVE

?Cl


(1)

VENT VALVE

?Cl


(1)

(1) HOT BUS supplies power during refueling on battery.
(2) This occurs if DC BUS 1 is lost.
(3) In smoke configuration (GEN 1 LINE pushbutton OFF), wing tank pumps 1 are supplied directly by IDG (instead of AC 1) and pump relays by DC ESS (instead of DC 1).


"""

question2 = ["What is the purpose of the main fuel pump"]


### Generate and print prediction

In [41]:
prediction2 = run_prediction(question2, context2)

for key in prediction2.keys():
  print(prediction2[key])

add example index and unique id: 100%|██████████| 1/1 [00:00<00:00, 3628.29it/s]


supplies fuel from the wing tanks to the engines


# References

1. Source Hugging Face Library from Github. Git checkout: Switch branches or restore working tree files<br>[Git Hub Repo](https://github.com/huggingface/transformers)
2. Get Training Data SQuAD 2.0 <br>[SQuAD Resource](https://rajpurkar.github.io/SQuAD-explorer/) 
3. ALBERT: Train Model <br>[Find Model Name](https://huggingface.co/transformers/pretrained_models.html) <br> [ALBERT Parameters](https://huggingface.co/transformers/model_doc/bert.html)
4. SentencePiece https://github.com/google/sentencepiece
5. Class SquadExample:<br>
    """<br>
    A single training/test example for the Squad dataset, as loaded from disk.
    Args:<br>
        qas_id: The example's unique identifier<br>
        question_text: The question string<br>
        context_text: The context string<br>
        answer_text: The answer string<br>
        start_position_character: The character position of the start of the <br>answer<br>
        title: The title of the example<br>
        answers: None by default, this is used during evaluation. Holds answers as well as their start positions.<br>
        is_impossible: False by default, set to True if the example has no possible answer.<br>
    """
    [Pytorch Documentation](https://pytorch.org/docs/stable/data.html)<br> [Model Outputs Documentation](https://huggingface.co/transformers/main_classes/output.html)